In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import math
import gc
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
import warnings
from SBBTree_ONLINE import SBBTree
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')


In [2]:
%run com_util.py

In [3]:
path = 'D:/learningdata/Mlearing/JData/B'
sku_basic_info=pd.read_csv(path+'/jdata_sku_basic_info.csv')
user_basic_info=pd.read_csv(path+'/jdata_user_basic_info.csv')
user_action=pd.read_csv(path+'/jdata_user_action.csv')
user_order=pd.read_csv(path+'/jdata_user_order.csv')
user_comment_score=pd.read_csv(path+'/jdata_user_comment_score.csv')
#sku_basic_info = sku_basic_info[(sku_basic_info.cate==30)|(sku_basic_info.cate==101)]
print (sku_basic_info.shape,user_basic_info.shape,user_action.shape,user_order.shape,user_comment_score.shape)


(108333, 6) (99446, 4) (8033923, 5) (881692, 6) (233366, 4)


In [4]:
####处理数据
user_action['a_date'] = pd.to_datetime(user_action['a_date'])
user_order['o_date'] = pd.to_datetime(user_order['o_date'])
user_comment_score['comment_create_tm'] = pd.to_datetime(user_comment_score['comment_create_tm'])

# sort by datetime
user_action = user_action.sort_values(['user_id','a_date'])
user_order = user_order.sort_values(['user_id','o_date'])
user_comment_score = user_comment_score.sort_values(['user_id','comment_create_tm'])

# year month day
user_order['o_year'] = user_order['o_date'].dt.year
user_order['o_month'] = user_order['o_date'].dt.month
user_order['o_day'] = user_order['o_date'].dt.day

user_action['a_year'] = user_action['a_date'].dt.year
user_action['a_month'] = user_action['a_date'].dt.month
user_action['a_day'] = user_action['a_date'].dt.day

user_comment_score['comment_year'] = user_comment_score['comment_create_tm'].dt.year
user_comment_score['comment_month'] = user_comment_score['comment_create_tm'].dt.month
user_comment_score['comment_day'] = user_comment_score['comment_create_tm'].dt.day

#Order Comment User Sku
Order_Comment_User_Sku = user_order.\
                            merge(user_comment_score,on=['user_id','o_id'],how='left').\
                            merge(user_basic_info,on='user_id',how='left').\
                            merge(sku_basic_info,on='sku_id',how='left')
# Action User Sku
Action_User_Sku = user_action.\
                        merge(user_basic_info,on='user_id',how='left').\
                        merge(sku_basic_info,on='sku_id',how='left')

Action_User_Sku = Action_User_Sku.drop(['age','sex','user_lv_cd','price','cate','para_1','para_2','para_3'],axis=1)

In [7]:
###dataset split :          其中dataset为提取label，，，feature为提取特征
###dataset3: 20170901-20170930         feature3:20170601-20170831  
###dataset2: 20170801-20170831         feature2:20170501-20170731
###dataset1: 20170701-20170731         featrue1:20170401-20170630

######此处有可能出现的问题就是当commentdate在时间窗口内的时候，o_date不在

dataset3_time = [datetime(2017,9,1),datetime(2017,9,30)]
dataset2_time = [datetime(2017,8,1),datetime(2017,8,31)]
dataset1_time = [datetime(2017,7,1),datetime(2017,7,31)]
feature3_time = [datetime(2017,6,1),datetime(2017,8,31)] 
feature2_time = [datetime(2017,5,1),datetime(2017,7,31)]
feature1_time = [datetime(2017,4,1),datetime(2017,6,30)]

feature1_OC = Order_Comment_User_Sku[((Order_Comment_User_Sku.o_date>=feature1_time[0])&(Order_Comment_User_Sku.o_date<=feature1_time[1]))|((Order_Comment_User_Sku.comment_create_tm>=feature1_time[0])&(Order_Comment_User_Sku.comment_create_tm<=feature1_time[1]))]
feature1_A = Action_User_Sku[(Action_User_Sku.a_date>=feature1_time[0])&(Action_User_Sku.a_date<=feature1_time[1])]
feature1 = feature1_OC.merge(feature1_A,on=['user_id','sku_id'],how='left')

feature2_OC = Order_Comment_User_Sku[((Order_Comment_User_Sku.o_date>=feature2_time[0])&(Order_Comment_User_Sku.o_date<=feature2_time[1]))|((Order_Comment_User_Sku.comment_create_tm>=feature2_time[0])&(Order_Comment_User_Sku.comment_create_tm<=feature2_time[1]))]
feature2_A = Action_User_Sku[(Action_User_Sku.a_date>=feature2_time[0])&(Action_User_Sku.a_date<=feature2_time[1])]
feature2 = feature2_OC.merge(feature2_A,on=['user_id','sku_id'],how='left')

feature3_OC = Order_Comment_User_Sku[((Order_Comment_User_Sku.o_date>=feature3_time[0])&(Order_Comment_User_Sku.o_date<=feature3_time[1]))|((Order_Comment_User_Sku.comment_create_tm>=feature3_time[0])&(Order_Comment_User_Sku.comment_create_tm<=feature3_time[1]))]
feature3_A = Action_User_Sku[(Action_User_Sku.a_date>=feature3_time[0])&(Action_User_Sku.a_date<=feature3_time[1])]
feature3 = feature3_OC.merge(feature3_A,on=['user_id','sku_id'],how='left')

In [8]:
###构造label
feature1 = feature1.fillna(0)
dataset1 = feature1[(featrue1.o_date!=0)|(feature.a_date!=0)|(feature.comment_create_tm!=0)]
dataset1 = dataset1[['sku_id','user_id','o_day','o_date']]
dataset1['label_1'] = 1
dataset1['label_2'] = dataset1['o_day']
dataset1 = dataset1[['sku_id','user_id','label_1','label_2']]
dataset1 = dataset1.drop_duplicates()


dataset2 = feature2[((feature2.o_date>=dataset2_time[0])&(feature2.o_date<=dataset2_time[1]))&((feature2.cate==30) | (feature2.cate==101))]
dataset2 = dataset2[['sku_id','user_id','o_day','o_date']]
dataset2['label_1'] = 1 
dataset2['label_2'] = dataset2['o_day']
dataset2 = dataset2[['sku_id','user_id','label_1','label_2']]
dataset2 = dataset2.drop_duplicates()
print(dataset2.shape,dataset1.shape)
gc.collect()

(0, 4) (0, 4)


218

In [ ]:
###商品特征:(sku_feature)

'''
预测目标:cate = {30,101}

自带特征：价格price，商品参数para_1，品类cate

在规定的时间窗口内：
    每个商品在时间窗口内被购买的次数m1
    M1/12此商品在京东7天的平均购买次数m2
    M1/6此商品在京东15天的平均购买次数m3
    M1/3此商品在京东31天的平均购买次数m4

    每个商品在时间窗口内被购买的总数m5
    M5/12此商品在京东7天的平均购买数目m6
    M5/6此商品在京东15天的平均购买数目m7
    M5/3此商品在京东31天的平均购买数目m8

    每个商品在时间窗口内被浏览的总次数m9
    M9/12此商品在京东7天的平均浏览数m10
    M9/6此商品在京东15天的平均浏览数m11
    M9/3此商品在京东31天的平均浏览数m12

    每个商品在时间窗口内被关注的总次数m13
    M13/12此商品在京东7天的平均浏览数m14
    M13/6此商品在京东15天的平均浏览数m15
    M13/3此商品在京东31天的平均浏览数m16

    每个商品在时间窗口内被评论的总次数m17
    M13/12此商品在京东7天的平均评论数m18
    M13/6此商品在京东15天的平均评论数m19
    M13/3此商品在京东31天的平均评论数m20


    
    此商品在时间窗口内被购买的平均时间间隔m21
    此商品在时间窗口内被浏览的平均时间间隔m22
    此商品在时间窗口内被关注的平均时间间隔m23
    此商品在时间窗口内被评论的平均时间间隔m24
    
    此商品在时间窗口内最早被购买的距离月初的平均时间间隔m25
    此商品在时间窗口内最早被浏览的距离月初的平均时间间隔m26
    此商品在时间窗口内最早被关注的距离月初的平均时间间隔m27
    此商品在时间窗口内最早被评论的距离月初的平均时间间隔m28
    
    此商品在时间窗口内最晚被购买的距离月末的平均时间间隔m29
    此商品在时间窗口内最晚被浏览的距离月末的平均时间间隔m30
    此商品在时间窗口内最晚被关注的距离月末的平均时间间隔m31
    此商品在时间窗口内最晚被评论的距离月末的平均时间间隔m32
    
    每个商品在时间窗口内被购买的总金额m33
    M33/12此商品在京东7天的平均消费金额m34
    M33/6此商品在京东15天的平均消费金额m35
    M33/3此商品在京东31天的平均消费金额m36
    
    此商品在时间窗口内最早购买的时间m37
    此商品在时间窗口内最早浏览的时间m38
    此商品在时间窗口内最早关注的时间m39
    此商品在时间窗口内最早评论的时间m40
    
    此商品在时间窗口内最晚购买的时间m41
    此商品在时间窗口内最晚浏览的时间m42
    此商品在时间窗口内最晚关注的时间m43
    此商品在时间窗口内最晚评论的时间m44

'''

In [8]:
###用户特征：(user_feature)

'''
自带特征：年龄age,用户等级user_lv_cd

在规定的时间窗口内:

    每个用户在时间窗口内购买的次数u1
    u1/12此用户在京东7天的平均购买次数u2
    u1/6此用户在京东15天的平均购买次数u3
    u1/3此用户在京东31天的平均购买次数u4

    每个用户在时间窗口内购买的总数u5
    u5/12此用户在京东7天的平均购买数目u6
    u5/6此用户在京东15天的平均购买数目u7
    u5/3此用户在京东31天的平均购买数目u8

    每个用户在时间窗口内浏览的总次数u9
    u9/12此用户在京东7天的平均浏览数u10
    u9/6此用户在京东15天的平均浏览数u11
    u9/3此用户在京东31天的平均浏览数u12

    每个用户在时间窗口内关注的总次数u13
    u13/12此用户在京东7天的平均浏览数u14
    u13/6此用户在京东15天的平均浏览数u15
    u13/3此用户在京东31天的平均浏览数u16

    每个用户在时间窗口内评论的总次数u13
    u13/12此用户在京东7天的平均评论数u14
    u13/6此用户在京东15天的平均评论数u15
    u13/3此用户在京东31天的平均评论数u16

    每个用户在时间窗口内购买的总金额u17
    U17/12此用户在京东7天的平均消费金额u18
    U13/6此用户在京东15天的平均消费金额u19
    U13/3此用户在京东31天的平均消费金额u20
    
    此用户在时间窗口内购买的平均时间间隔u21
    此用户在时间窗口内浏览的平均时间间隔u21
    此用户在时间窗口内关注的平均时间间隔u22
    此用户在时间窗口内评论的平均时间间隔u23

    此用户在时间窗口内最早购买的距离月初的平均时间间隔u24
    此用户在时间窗口内最早浏览的距离月初的平均时间间隔u25
    此用户在时间窗口内最早关注的距离月初的平均时间间隔u26
    此用户在时间窗口内最早评论的距离月初的平均时间间隔u27
    
    此用户在时间窗口内最晚购买的距离月末的平均时间间隔u28
    此用户在时间窗口内最晚浏览的距离月末的平均时间间隔u29
    此用户在时间窗口内最晚关注的距离月末的平均时间间隔u30
    此用户在时间窗口内最晚评论的距离月末的平均时间间隔u31
    
    每个用户在时间窗口内购买的总金额u33
    u33/12此用户在京东7天的平均消费金额u34
    u33/6此用户在京东15天的平均消费金额u35
    u33/3此用户在京东31天的平均消费金额u36

    此用户在时间窗口内最早购买的时间u37
    此用户在时间窗口内最早浏览的时间u38
    此用户在时间窗口内最早关注的时间u39
    此用户在时间窗口内最早评论的时间u40
    
    此用户在时间窗口内最晚购买的时间u41
    此用户在时间窗口内最晚浏览的时间u42
    此用户在时间窗口内最晚关注的时间u43
    此用户在时间窗口内最晚评论的时间u44

'''

'\n自带特征：年龄age,用户等级user_lv_cd\n\n在规定的时间窗口内:\n\n    每个用户在时间窗口内购买的次数u1\n    u1/12此用户在京东7天的平均购买次数u2\n    u1/6此用户在京东15天的平均购买次数u3\n    u1/3此用户在京东31天的平均购买次数u4\n\n    每个用户在时间窗口内购买的总数u5\n    u5/12此用户在京东7天的平均购买数目u6\n    u5/6此用户在京东15天的平均购买数目u7\n    u5/3此用户在京东31天的平均购买数目u8\n\n    每个用户在时间窗口内浏览的总次数u9\n    u9/12此用户在京东7天的平均浏览数u10\n    u9/6此用户在京东15天的平均浏览数u11\n    u9/3此用户在京东31天的平均浏览数u12\n\n    每个用户在时间窗口内关注的总次数u13\n    u13/12此用户在京东7天的平均浏览数u14\n    u13/6此用户在京东15天的平均浏览数u15\n    u13/3此用户在京东31天的平均浏览数u16\n\n    每个用户在时间窗口内评论的总次数u13\n    u13/12此用户在京东7天的平均评论数u14\n    u13/6此用户在京东15天的平均评论数u15\n    u13/3此用户在京东31天的平均评论数u16\n\n    每个用户在时间窗口内购买的总金额u17\n    U17/12此用户在京东7天的平均消费金额u18\n    U13/6此用户在京东15天的平均消费金额u19\n    U13/3此用户在京东31天的平均消费金额u20\n    \n    此用户在时间窗口内购买的平均时间间隔u21\n    此用户在时间窗口内浏览的平均时间间隔u21\n    此用户在时间窗口内关注的平均时间间隔u22\n    此用户在时间窗口内评论的平均时间间隔u23\n\n    此用户在时间窗口内最早购买的距离月初的平均时间间隔u24\n    此用户在时间窗口内最早浏览的距离月初的平均时间间隔u25\n    此用户在时间窗口内最早关注的距离月初的平均时间间隔u26\n    此用户在时间窗口内最早评论的距离月初的平均时间间隔u27\n    \n    此用户在时间窗口内最晚购买的距离月末的平均

In [9]:
#####商品用户特征(user_sku_feature)

'''
在规定的时间窗口内:

    用户对此商品在时间窗口内购买的次数um1
    UM1/12此用户对此商品在京东7天的平均购买次数um2
    UM1/6此用户对此商品在京东15天的平均购买次数um3
    UM1/3此用户对此商品在京东31天的平均购买次数um4

    用户对此商品在时间窗口内购买的总数um5
    UM5/12此用户对此商品在京东7天的平均购买数目um6
    UM5/6此用户对此商品在京东15天的平均购买数目um7
    UM5/3此用户对此商品在京东31天的平均购买数目um8

    用户在对此商品时间窗口内浏览的总次数um9
    UM9/12此用户对此商品在京东7天的平均浏览数um10
    UM9/6此用户对此商品在京东15天的平均浏览数um11
    UM9/3此用户对此商品在京东31天的平均浏览数um12

    用户对此商品在时间窗口内关注的总次数um13
    UM13/12用户对此商品在京东7天的平均浏览数um14
    UM13/6用户对此商品在京东15天的平均浏览数um15
    UM13/3用户对此商品在京东31天的平均浏览数um16

    用户对此商品在时间窗口内评论的总次数um13
    UM13/12此用户对此商品在京东7天的平均评论数um14
    UM13/6此用户对此商品在京东15天的平均评论数um15
    UM13/3此用户对此商品在京东31天的平均评论数um16

    用户对此商品在时间窗口内购买的总金额um17
    UM17/12此用户对此商品在京东7天的平均消费金额um18
    UM13/6此用户对此商品在京东15天的平均消费金额um19
    UM13/3此用户对此商品在京东31天的平均消费金额um20
    
    此用户对此商品在时间窗口内购买的平均时间间隔um21
    此用户对此商品在时间窗口内浏览的平均时间间隔um21
    此用户对此商品在时间窗口内关注的平均时间间隔um22
    此用户对此商品在时间窗口内评论的平均时间间隔um23

    此用户在时间窗口内对此商品最早购买的距离月初的平均时间间隔um24
    此用户在时间窗口内对此商品最早浏览的距离月初的平均时间间隔um25
    此用户在时间窗口内对此商品最早关注的距离月初的平均时间间隔um26
    此用户在时间窗口内对此商品最早评论的距离月初的平均时间间隔um27
    
    此用户在时间窗口内对此商品最晚购买的距离月末的平均时间间隔um28
    此用户在时间窗口内对此商品最晚浏览的距离月末的平均时间间隔um29
    此用户在时间窗口内对此商品最晚关注的距离月末的平均时间间隔um30
    此用户在时间窗口内对此商品最晚评论的距离月末的平均时间间隔um31

    用户在时间窗口内对该商品购买的总金额um33
    M17/12此用户在京东7天该商品的平均消费金额um34
    M13/6此用户在商品在京东15天该商品的平均消费金额um35
    M13/3此用户在商品在京东31天该商品的平均消费金额um36
    
    此用户对此商品在时间窗口内最早购买的时间u37
    此用户对此商品在时间窗口内最早浏览的时间u38
    此用户对此商品在时间窗口内最早关注的时间u39
    此用户对此商品在时间窗口内最早评论的时间u40
    
    此用户对此商品在时间窗口内最晚购买的时间u41
    此用户对此商品在时间窗口内最晚浏览的时间u42
    此用户对此商品在时间窗口内最晚关注的时间u43
    此用户对此商品在时间窗口内最晚评论的时间u44
'''

'\n在规定的时间窗口内:\n\n    用户对此商品在时间窗口内购买的次数um1\n    UM1/12此用户对此商品在京东7天的平均购买次数um2\n    UM1/6此用户对此商品在京东15天的平均购买次数um3\n    UM1/3此用户对此商品在京东31天的平均购买次数um4\n\n    用户对此商品在时间窗口内购买的总数um5\n    UM5/12此用户对此商品在京东7天的平均购买数目um6\n    UM5/6此用户对此商品在京东15天的平均购买数目um7\n    UM5/3此用户对此商品在京东31天的平均购买数目um8\n\n    用户在对此商品时间窗口内浏览的总次数um9\n    UM9/12此用户对此商品在京东7天的平均浏览数um10\n    UM9/6此用户对此商品在京东15天的平均浏览数um11\n    UM9/3此用户对此商品在京东31天的平均浏览数um12\n\n    用户对此商品在时间窗口内关注的总次数um13\n    UM13/12用户对此商品在京东7天的平均浏览数um14\n    UM13/6用户对此商品在京东15天的平均浏览数um15\n    UM13/3用户对此商品在京东31天的平均浏览数um16\n\n    用户对此商品在时间窗口内评论的总次数um13\n    UM13/12此用户对此商品在京东7天的平均评论数um14\n    UM13/6此用户对此商品在京东15天的平均评论数um15\n    UM13/3此用户对此商品在京东31天的平均评论数um16\n\n    用户对此商品在时间窗口内购买的总金额um17\n    UM17/12此用户对此商品在京东7天的平均消费金额um18\n    UM13/6此用户对此商品在京东15天的平均消费金额um19\n    UM13/3此用户对此商品在京东31天的平均消费金额um20\n    \n    此用户对此商品在时间窗口内购买的平均时间间隔um21\n    此用户对此商品在时间窗口内浏览的平均时间间隔um21\n    此用户对此商品在时间窗口内关注的平均时间间隔um22\n    此用户对此商品在时间窗口内评论的平均时间间隔um23\n\n    此用户在时间窗口内对此商品最早购买的距离月初的平均时间间隔um24\n    此用户在时间窗口内对

In [12]:
####商品特征
#######dataset3_sku
#######
####for dataset3
sku3 = feature3[['sku_id','o_id','o_date','o_sku_num','o_month','o_day','comment_create_tm','score_level','comment_year','comment_month','comment_day','price','cate','para_1','a_date','a_num','a_type','a_year','a_month','a_day']]
t = sku3[['sku_id','price','cate']]
t = t[(t.cate==30)|(t.cate==101)]

t.drop_duplicates(inplace=True)
###m1
sample_sku3 = feat_nunique(t,sku3[(sku3.cate==101)|(sku3.cate==30)],['sku_id'],'o_id','f3_buytimes_30101_')
sample_sku3 = feat_nunique(sample_sku3,sku3[(sku3.cate==30)],['sku_id'],'o_id','f3_buytimes_30_')
sample_sku3 = feat_nunique(sample_sku3,sku3[(sku3.cate==101)],['sku_id'],'o_id','f3_buytimes_101_')
sample_sku3 = feat_nunique(sample_sku3,sku3[(sku3.cate!=101)&(sku3.cate!=30)],['sku_id'],'o_id','f3_buytimes_no_30101_')

###m2-m4
sample_sku3['f3_buytimes_30101_in_7days_'] = sample_sku3['f3_buytimes_30101_o_id_sku_id_nunique']/12
sample_sku3['f3_buytimes_30_in_7days_'] = sample_sku3['f3_buytimes_30_o_id_sku_id_nunique']/12
sample_sku3['f3_buytimes_101_in_7days_'] = sample_sku3['f3_buytimes_101_o_id_sku_id_nunique']/12
sample_sku3['f3_buytimes_no_30101_in_7days_'] = sample_sku3['f3_buytimes_no_30101_o_id_sku_id_nunique']/12

sample_sku3['f3_buytimes_30101_in_15days_'] = sample_sku3['f3_buytimes_30101_o_id_sku_id_nunique']/6
sample_sku3['f3_buytimes_30_in_15days_'] = sample_sku3['f3_buytimes_30_o_id_sku_id_nunique']/6
sample_sku3['f3_buytimes_101_in_15days_'] = sample_sku3['f3_buytimes_101_o_id_sku_id_nunique']/6
sample_sku3['f3_buytimes_no_30101_in_15days_'] = sample_sku3['f3_buytimes_no_30101_o_id_sku_id_nunique']/6

sample_sku3['f3_buytimes_30101_in_31days_'] = sample_sku3['f3_buytimes_30101_o_id_sku_id_nunique']/3
sample_sku3['f3_buytimes_30_in_31days_'] = sample_sku3['f3_buytimes_30_o_id_sku_id_nunique']/3
sample_sku3['f3_buytimes_101_in_31days_'] = sample_sku3['f3_buytimes_101_o_id_sku_id_nunique']/3
sample_sku3['f3_buytimes_no_30101_in_31days_'] = sample_sku3['f3_buytimes_no_30101_o_id_sku_id_nunique']/3

###m5
sample_sku3 = feat_sum(sample_sku3,sku3[(sku3.cate==101)|(sku3.cate==30)],['sku_id'],'o_sku_num','f3_buynums_30101_')
sample_sku3 = feat_sum(sample_sku3,sku3[(sku3.cate==30)],['sku_id'],'o_sku_num','f3_buynums_30_')
sample_sku3 = feat_sum(sample_sku3,sku3[(sku3.cate==101)],['sku_id'],'o_sku_num','f3_buynums_101_')
sample_sku3 = feat_sum(sample_sku3,sku3[(sku3.cate!=101)&(sku3.cate!=30)],['sku_id'],'o_sku_num','f3_buynums_no_30101_')

###m6-m8
sample_sku3['f3_buynums_30101_in_7days_'] = sample_sku3['f3_buynums_30101_o_sku_num_sku_id_sum']/12
sample_sku3['f3_buynums_30_in_7days_'] = sample_sku3['f3_buynums_30_o_sku_num_sku_id_sum']/12
sample_sku3['f3_buynums_101_in_7days_'] = sample_sku3['f3_buynums_101_o_sku_num_sku_id_sum']/12
sample_sku3['f3_buynums_no_30101_in_7days_'] = sample_sku3['f3_buynums_no_30101_o_sku_num_sku_id_sum']/12

sample_sku3['f3_buynums_30101_in_15days_'] = sample_sku3['f3_buynums_30101_o_sku_num_sku_id_sum']/6
sample_sku3['f3_buynums_30_in_15days_'] = sample_sku3['f3_buynums_30_o_sku_num_sku_id_sum']/6
sample_sku3['f3_buynums_101_in_15days_'] = sample_sku3['f3_buynums_101_o_sku_num_sku_id_sum']/6
sample_sku3['f3_buynums_no_30101_in_15days_'] = sample_sku3['f3_buynums_no_30101_o_sku_num_sku_id_sum']/6

sample_sku3['f3_buynums_30101_in_31days_'] = sample_sku3['f3_buynums_30101_o_sku_num_sku_id_sum']/3
sample_sku3['f3_buynums_30_in_31days_'] = sample_sku3['f3_buynums_30_o_sku_num_sku_id_sum']/3
sample_sku3['f3_buynums_101_in_31days_'] = sample_sku3['f3_buynums_101_o_sku_num_sku_id_sum']/3
sample_sku3['f3_buynums_no_30101_in_31days_'] = sample_sku3['f3_buynums_no_30101_o_sku_num_sku_id_sum']/3

###m9
sample_sku3 = feat_sum(sample_sku3,sku3[((sku3.cate==101)|(sku3.cate==30))&(sku3.a_type==1)],['sku_id'],'a_num','f3_scannums_30101_')
sample_sku3 = feat_sum(sample_sku3,sku3[((sku3.cate==30))&(sku3.a_type==1)],['sku_id'],'a_num','f3_scannums_30_')
sample_sku3 = feat_sum(sample_sku3,sku3[((sku3.cate==101))&(sku3.a_type==1)],['sku_id'],'a_num','f3_scannums_101_')
sample_sku3 = feat_sum(sample_sku3,sku3[((sku3.cate!=101)&(sku3.cate!=30))&(sku3.a_type==1)],['sku_id'],'a_num','f3_scannums_no_30101_')
###m10-m12
sample_sku3['f3_scannums_30101_in_7days_'] = sample_sku3['f3_scannums_30101_a_num_sku_id_sum']/12
sample_sku3['f3_scannums_30_in_7days_'] = sample_sku3['f3_scannums_30_a_num_sku_id_sum']/12
sample_sku3['f3_scannums_101_in_7days_'] = sample_sku3['f3_scannums_101_a_num_sku_id_sum']/12
sample_sku3['f3_scannums_no_30101_in_7days_'] = sample_sku3['f3_scannums_no_30101_a_num_sku_id_sum']/12

sample_sku3['f3_scannums_30101_in_15days_'] = sample_sku3['f3_scannums_30101_a_num_sku_id_sum']/6
sample_sku3['f3_scannums_30_in_15days_'] = sample_sku3['f3_scannums_30_a_num_sku_id_sum']/6
sample_sku3['f3_scannums_101_in_15days_'] = sample_sku3['f3_scannums_101_a_num_sku_id_sum']/6
sample_sku3['f3_scannums_no_30101_in_15days_'] = sample_sku3['f3_scannums_no_30101_a_num_sku_id_sum']/6

sample_sku3['f3_scannums_30101_in_31days_'] = sample_sku3['f3_scannums_30101_a_num_sku_id_sum']/3
sample_sku3['f3_scannums_30_in_31days_'] = sample_sku3['f3_scannums_30_a_num_sku_id_sum']/3
sample_sku3['f3_scannums_101_in_31days_'] = sample_sku3['f3_scannums_101_a_num_sku_id_sum']/3
sample_sku3['f3_scannums_no_30101_in_31days_'] = sample_sku3['f3_scannums_no_30101_a_num_sku_id_sum']/3


###m13
sample_sku3 = feat_sum(sample_sku3,sku3[((sku3.cate==101)|(sku3.cate==30))&(sku3.a_type==2)],['sku_id'],'a_num','f3_attnums_30101_')
sample_sku3 = feat_sum(sample_sku3,sku3[((sku3.cate==30))&(sku3.a_type==2)],['sku_id'],'a_num','f3_attnums_30_')
sample_sku3 = feat_sum(sample_sku3,sku3[((sku3.cate==101))&(sku3.a_type==2)],['sku_id'],'a_num','f3_attnums_101_')
sample_sku3 = feat_sum(sample_sku3,sku3[((sku3.cate!=101)&(sku3.cate!=30))&(sku3.a_type==2)],['sku_id'],'a_num','f3_attnums_no_30101_')
###m14-m16
sample_sku3['f3_attnums_30101_in_7days_'] = sample_sku3['f3_attnums_30101_a_num_sku_id_sum']/12
sample_sku3['f3_attnums_30_in_7days_'] = sample_sku3['f3_attnums_30_a_num_sku_id_sum']/12
sample_sku3['f3_attnums_101_in_7days_'] = sample_sku3['f3_attnums_101_a_num_sku_id_sum']/12
sample_sku3['f3_attnums_no_30101_in_7days_'] = sample_sku3['f3_attnums_no_30101_a_num_sku_id_sum']/12

sample_sku3['f3_attnums_30101_in_15days_'] = sample_sku3['f3_attnums_30101_a_num_sku_id_sum']/6
sample_sku3['f3_attnums_30_in_15days_'] = sample_sku3['f3_attnums_30_a_num_sku_id_sum']/6
sample_sku3['f3_attnums_101_in_15days_'] = sample_sku3['f3_attnums_101_a_num_sku_id_sum']/6
sample_sku3['f3_attnums_no_30101_in_15days_'] = sample_sku3['f3_attnums_no_30101_a_num_sku_id_sum']/6

sample_sku3['f3_attnums_30101_in_31days_'] = sample_sku3['f3_attnums_30101_a_num_sku_id_sum']/3
sample_sku3['f3_attnums_30_in_31days_'] = sample_sku3['f3_attnums_30_a_num_sku_id_sum']/3
sample_sku3['f3_attnums_101_in_31days_'] = sample_sku3['f3_attnums_101_a_num_sku_id_sum']/3
sample_sku3['f3_attnums_no_30101_in_31days_'] = sample_sku3['f3_attnums_no_30101_a_num_sku_id_sum']/3



###m17
sample_sku3 = feat_nunique(sample_sku3,sku3[(sku3.cate==101)|(sku3.cate==30)],['sku_id'],'comment_create_tm','f3_commentnums_30101_')
sample_sku3 = feat_nunique(sample_sku3,sku3[(sku3.cate==30)],['sku_id'],'comment_create_tm','f3_commentnums_30_')
sample_sku3 = feat_nunique(sample_sku3,sku3[(sku3.cate==101)],['sku_id'],'comment_create_tm','f3_commentnums_101_')
sample_sku3 = feat_nunique(sample_sku3,sku3[(sku3.cate!=101)&(sku3.cate!=30)],['sku_id'],'comment_create_tm','f3_commentnums_no_30101_')

###m18-20
sample_sku3['f3_commentnums_30101_in_7days_'] = sample_sku3['f3_commentnums_30101_comment_create_tm_sku_id_nunique']/12
sample_sku3['f3_commentnums_30_in_7days_'] = sample_sku3['f3_commentnums_30_comment_create_tm_sku_id_nunique']/12
sample_sku3['f3_commentnums_101_in_7days_'] = sample_sku3['f3_commentnums_101_comment_create_tm_sku_id_nunique']/12
sample_sku3['f3_commentnums_no_30101_in_7days_'] = sample_sku3['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']/12

sample_sku3['f3_commentnums_30101_in_15days_'] = sample_sku3['f3_commentnums_30101_comment_create_tm_sku_id_nunique']/6
sample_sku3['f3_commentnums_30_in_15days_'] = sample_sku3['f3_commentnums_30_comment_create_tm_sku_id_nunique']/6
sample_sku3['f3_commentnums_101_in_15days_'] = sample_sku3['f3_commentnums_101_comment_create_tm_sku_id_nunique']/6
sample_sku3['f3_commentnums_no_30101_in_15days_'] = sample_sku3['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']/6

sample_sku3['f3_commentnums_30101_in_31days_'] = sample_sku3['f3_commentnums_30101_comment_create_tm_sku_id_nunique']/3
sample_sku3['f3_commentnums_30_in_31days_'] = sample_sku3['f3_commentnums_30_comment_create_tm_sku_id_nunique']/3
sample_sku3['f3_commentnums_101_in_31days_'] = sample_sku3['f3_commentnums_101_comment_create_tm_sku_id_nunique']/3
sample_sku3['f3_commentnums_no_30101_in_31days_'] = sample_sku3['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']/3

###m21
t1 = sample_sku3[['sku_id','f3_buytimes_30101_o_id_sku_id_nunique']]
t2 = sku3[(sku3.cate==101)|(sku3.cate==30)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.cate==101)|(sku3.cate==30)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_30101_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_30101_buy_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_sku3[['sku_id','f3_buytimes_30_o_id_sku_id_nunique']]
t2 = sku3[(sku3.cate==30)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.cate==30)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_30_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_30_buy_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_sku3[['sku_id','f3_buytimes_101_o_id_sku_id_nunique']]
t2 = sku3[(sku3.cate==101)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.cate==101)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_101_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_101_buy_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_sku3[['sku_id','f3_buytimes_no_30101_o_id_sku_id_nunique']]
t2 = sku3[(sku3.cate!=101)&(sku3.cate!=30)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.cate!=101)&(sku3.cate!=30)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_no_30101_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_no_30101_buy_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')

###m22
t1 = sample_sku3[['sku_id','f3_scannums_30101_a_num_sku_id_sum']]
t2 = sku3[(sku3.a_type==1)&((sku3.cate==101)|(sku3.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.a_type==1)&((sku3.cate==101)|(sku3.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30101_scan_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_sku3[['sku_id','f3_scannums_30_a_num_sku_id_sum']]
t2 = sku3[(sku3.a_type==1)&((sku3.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.a_type==1)&((sku3.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_30_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30_scan_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_sku3[['sku_id','f3_scannums_101_a_num_sku_id_sum']]
t2 = sku3[(sku3.a_type==1)&((sku3.cate==101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.a_type==1)&((sku3.cate==101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_101_scan_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_sku3[['sku_id','f3_scannums_no_30101_a_num_sku_id_sum']]
t2 = sku3[(sku3.a_type==1)&((sku3.cate!=30)&(sku3.cate!=101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.a_type==1)&((sku3.cate!=30)&(sku3.cate!=101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_no_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_no_30101_scan_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')


###m23
t1 = sample_sku3[['sku_id','f3_attnums_30101_a_num_sku_id_sum']]
t2 = sku3[(sku3.a_type==2)&((sku3.cate==101)|(sku3.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.a_type==2)&((sku3.cate==101)|(sku3.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30101_att_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_sku3[['sku_id','f3_attnums_30_a_num_sku_id_sum']]
t2 = sku3[(sku3.a_type==2)&((sku3.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.a_type==2)&((sku3.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_30_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30_att_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_sku3[['sku_id','f3_attnums_101_a_num_sku_id_sum']]
t2 = sku3[(sku3.a_type==2)&((sku3.cate==101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.a_type==2)&((sku3.cate==101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_101_att_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_sku3[['sku_id','f3_attnums_no_30101_a_num_sku_id_sum']]
t2 = sku3[(sku3.a_type==2)&((sku3.cate!=30)&(sku3.cate!=101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.a_type==2)&((sku3.cate!=30)&(sku3.cate!=101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_no_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_no_30101_att_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')


###m24
t1 = sample_sku3[['sku_id','f3_commentnums_30101_comment_create_tm_sku_id_nunique']]
t2 = sku3[(sku3.cate==101)|(sku3.cate==30)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.cate==101)|(sku3.cate==30)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_30101_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_30101_comment_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')

t1 = sample_sku3[['sku_id','f3_commentnums_30_comment_create_tm_sku_id_nunique']]
t2 = sku3[(sku3.cate==30)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.cate==30)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_30_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_30_comment_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')


t1 = sample_sku3[['sku_id','f3_commentnums_101_comment_create_tm_sku_id_nunique']]
t2 = sku3[(sku3.cate==101)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.cate==101)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_101_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_101_comment_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')


t1 = sample_sku3[['sku_id','f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']]
t2 = sku3[(sku3.cate!=101)&(sku3.cate!=30)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[(sku3.cate!=101)&(sku3.cate!=30)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_no_30101_comment_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')

###m25-m32还没写，正在考虑
sku3_a_o_date = sku3[((pd.to_datetime(sku3.a_date)<=pd.to_datetime(sku3.o_date.fillna('2018-01-01'))))]
sku3_a_o_date = sku3_a_o_date.sort_values(by=['sku_id','a_date','o_date']).drop_duplicates(['sku_id','a_date'],keep="first")
sku3_a_o_date = sku3_a_o_date[['sku_id','a_date','o_date','cate']]
sku3_a_o_date['o_a_gap']=(pd.to_datetime(sku3_a_o_date.o_date)-pd.to_datetime(sku3_a_o_date.a_date)).dt.days 
FLAG='sku3_a_o_'
sample_sku3 = feat_count(sample_sku3,sku3_a_o_date[sku3_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG+'order_')
sample_sku3 = feat_mean(sample_sku3,sku3_a_o_date[(sku3_a_o_date.o_a_gap>=-1)&((sku3_a_o_date.cate==101)|(sku3_a_o_date.cate==30))],['sku_id'],'o_a_gap',FLAG+'30101_')
sample_sku3 = feat_mean(sample_sku3,sku3_a_o_date[(sku3_a_o_date.o_a_gap>=-1)&((sku3_a_o_date.cate==30))],['sku_id'],'o_a_gap',FLAG+'30_')
sample_sku3 = feat_mean(sample_sku3,sku3_a_o_date[(sku3_a_o_date.o_a_gap>=-1)&((sku3_a_o_date.cate==101))],['sku_id'],'o_a_gap',FLAG+'101_')
sample_sku3 = feat_mean(sample_sku3,sku3_a_o_date[(sku3_a_o_date.o_a_gap>=-1)&((sku3_a_o_date.cate!=101)&(sku3_a_o_date.cate!=30))],['sku_id'],'o_a_gap',FLAG+'no_30101_')




###m33
t1 = sample_sku3[['sku_id','f3_buynums_o_sku_num_sku_id_sum']] 
t2 = sku3[['sku_id','price']]
t3 = t1.merge(t2,on=['sku_id'],how='left')
t3['f3_total_price'] = t3['f3_buynums_o_sku_num_sku_id_sum']*t3['price']
t3 = t3[['sku_id','f3_total_price']]
sample_sku3 = sample_sku3.merge(t3,on=['sku_id'],how='left')

###m34-m36
sample_sku3['f3_total_price_in_7days_'] = sample_sku3['f3_total_price']/12
sample_sku3['f3_total_price_in_15days_'] = sample_sku3['f3_total_price']/6
sample_sku3['f3_total_price_in_31days_'] = sample_sku3['f3_total_price']/3

###m37-m40
sample_sku3 = feat_min(sample_sku3,sku3,['sku_id'],'o_day','f3_buytime_early_')
sample_sku3 = feat_min(sample_sku3,sku3[sku3.a_type==1],['sku_id'],'a_day','f3_scantime_early_')
sample_sku3 = feat_min(sample_sku3,sku3[sku3.a_type==2],['sku_id'],'a_day','f3_atttime_early_')
sample_sku3 = feat_min(sample_sku3,sku3,['sku_id'],'comment_day','f3_commenttime_early_')

###m41-m44
sample_sku3 = feat_max(sample_sku3,sku3,['sku_id'],'o_day','f3_buytime_late_')
sample_sku3 = feat_max(sample_sku3,sku3[sku3.a_type==1],['sku_id'],'a_day','f3_scantime_late_')
sample_sku3 = feat_max(sample_sku3,sku3[sku3.a_type==2],['sku_id'],'a_day','f3_atttime_late_')
sample_sku3 = feat_max(sample_sku3,sku3,['sku_id'],'comment_day','f3_commenttime_late_')

###
sample_sku3 = sample_sku3.drop_duplicates()
gc.collect()


KeyError: 'f3_buytimes_30101_o_id_sku_id_nunique'

In [ ]:
sku3 = feature3[['sku_id','o_id','o_date','o_sku_num','o_month','o_day','comment_create_tm','score_level','comment_year','comment_month','comment_day','price','cate','para_1','a_date','a_num','a_type','a_year','a_month','a_day']]
t = sku3[['sku_id','price','cate']]
t = t[(t.cate==30)|(t.cate==101)]

t.drop_duplicates(inplace=True)
###m1
sample_sku3 = feat_nunique(t,sku3,['sku_id'],'o_id','f3_buytimes_')
###m2-m4
sample_sku3['f3_buytimes_in_7days_'] = sample_sku3['f3_buytimes_o_id_sku_id_nunique']/12
sample_sku3['f3_buytimes_in_15days_'] = sample_sku3['f3_buytimes_o_id_sku_id_nunique']/6
sample_sku3['f3_buytimes_in_31days_'] = sample_sku3['f3_buytimes_o_id_sku_id_nunique']/3

###m5
sample_sku3 = feat_sum(sample_sku3,sku3,['sku_id'],'o_sku_num','f3_buynums_')
###m6-m8
sample_sku3['f3_buynums_in_7days_'] = sample_sku3['f3_buynums_o_sku_num_sku_id_sum']/12
sample_sku3['f3_buynums_in_15days_'] = sample_sku3['f3_buynums_o_sku_num_sku_id_sum']/6
sample_sku3['f3_buynums_in_31days_'] = sample_sku3['f3_buynums_o_sku_num_sku_id_sum']/3

###m9
sample_sku3 = feat_sum(sample_sku3,sku3[sku3.a_type==1],['sku_id'],'a_num','f3_scannums_')
###m10-m12
sample_sku3['f3_scannums_in_7days_'] = sample_sku3['f3_scannums_a_num_sku_id_sum']/12
sample_sku3['f3_scannums_in_15days_'] = sample_sku3['f3_scannums_a_num_sku_id_sum']/6
sample_sku3['f3_scannums_in_31days_'] = sample_sku3['f3_scannums_a_num_sku_id_sum']/3

###m13
sample_sku3 = feat_sum(sample_sku3,sku3[sku3.a_type==2],['sku_id'],'a_num','f3_attnums_')
###m14-m16
sample_sku3['f3_attnums_in_7days_'] = sample_sku3['f3_attnums_a_num_sku_id_sum']/12
sample_sku3['f3_attnums_in_15days_'] = sample_sku3['f3_attnums_a_num_sku_id_sum']/6
sample_sku3['f3_attnums_in_31days_'] = sample_sku3['f3_attnums_a_num_sku_id_sum']/3

###m17
sample_sku3 = feat_nunique(sample_sku3,sku3,['sku_id'],'comment_create_tm','f3_commentnums_')
###m18-20
sample_sku3['f3_commentnums_in_7days_'] = sample_sku3['f3_commentnums_comment_create_tm_sku_id_nunique']/12
sample_sku3['f3_commentnums_in_15days_'] = sample_sku3['f3_commentnums_comment_create_tm_sku_id_nunique']/6
sample_sku3['f3_commentnums_in_31days_'] = sample_sku3['f3_commentnums_comment_create_tm_sku_id_nunique']/3

###m21
t1 = sample_sku3[['sku_id','f3_buytimes_o_id_sku_id_nunique']]
t2 = sku3.groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3.groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_buy_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')

###m22
t1 = sample_sku3[['sku_id','f3_scannums_a_num_sku_id_sum']]
t2 = sku3[sku3.a_type==1].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[sku3.a_type==1].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_a_num_sku_id_sum']
t = t[['sku_id','f3_average_scan_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')
sample_sku3

###m23
t1 = sample_sku3[['sku_id','f3_attnums_a_num_sku_id_sum']]
t2 = sku3[sku3.a_type==2].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3[sku3.a_type==2].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_a_num_sku_id_sum']
t = t[['sku_id','f3_average_att_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')


###m24
t1 = sample_sku3[['sku_id','f3_commentnums_comment_create_tm_sku_id_nunique']]
t2 = sku3.groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku3.groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_comment_day_gap']]
sample_sku3 = sample_sku3.merge(t,on=['sku_id'],how='left')


###m25-m32还没写，正在考虑
sku3_a_o_date = sku3[((pd.to_datetime(sku3.a_date)<=pd.to_datetime(sku3.o_date.fillna('2018-01-01'))))]
sku3_a_o_date = sku3_a_o_date.sort_values(by=['sku_id','a_date','o_date']).drop_duplicates(['sku_id','a_date'],keep="first")
sku3_a_o_date = sku3_a_o_date[['sku_id','a_date','o_date','cate']]
sku3_a_o_date['o_a_gap']=(pd.to_datetime(sku3_a_o_date.o_date)-pd.to_datetime(sku3_a_o_date.a_date)).dt.days 
FLAG='sku3_a_o_'
sample_sku3 = feat_count(sample_sku3,sku3_a_o_date[sku3_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG+'order_')
sample_sku3 = feat_mean(sample_sku3,sku3_a_o_date[sku3_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG)

###m33
t1 = sample_sku3[['sku_id','f3_buynums_o_sku_num_sku_id_sum']] 
t2 = sku3[['sku_id','price']]
t3 = t1.merge(t2,on=['sku_id'],how='left')
t3['f3_total_price'] = t3['f3_buynums_o_sku_num_sku_id_sum']*t3['price']
t3 = t3[['sku_id','f3_total_price']]
sample_sku3 = sample_sku3.merge(t3,on=['sku_id'],how='left')

###m34-m36
sample_sku3['f3_total_price_in_7days_'] = sample_sku3['f3_total_price']/12
sample_sku3['f3_total_price_in_15days_'] = sample_sku3['f3_total_price']/6
sample_sku3['f3_total_price_in_31days_'] = sample_sku3['f3_total_price']/3

###m37-m40
sample_sku3 = feat_min(sample_sku3,sku3,['sku_id'],'o_day','f3_buytime_early_')
sample_sku3 = feat_min(sample_sku3,sku3[sku3.a_type==1],['sku_id'],'a_day','f3_scantime_early_')
sample_sku3 = feat_min(sample_sku3,sku3[sku3.a_type==2],['sku_id'],'a_day','f3_atttime_early_')
sample_sku3 = feat_min(sample_sku3,sku3,['sku_id'],'comment_day','f3_commenttime_early_')

###m41-m44
sample_sku3 = feat_max(sample_sku3,sku3,['sku_id'],'o_day','f3_buytime_late_')
sample_sku3 = feat_max(sample_sku3,sku3[sku3.a_type==1],['sku_id'],'a_day','f3_scantime_late_')
sample_sku3 = feat_max(sample_sku3,sku3[sku3.a_type==2],['sku_id'],'a_day','f3_atttime_late_')
sample_sku3 = feat_max(sample_sku3,sku3,['sku_id'],'comment_day','f3_commenttime_late_')

###
sample_sku3 = sample_sku3.drop_duplicates()
gc.collect()

In [17]:
####商品特征
#######dataset2_sku
#######
####for dataset2
sku2 = feature2[['sku_id','o_id','o_date','o_sku_num','o_month','o_day','comment_create_tm','score_level','comment_year','comment_month','comment_day','price','cate','para_1','a_date','a_num','a_type','a_year','a_month','a_day']]
t = sku2[['sku_id','price','cate','a_date','o_date']]
t = sku2[(sku2.cate==30)|(sku2.cate==101)]
t.drop_duplicates(inplace=True)
###m1
sample_sku2 = feat_nunique(t,sku2,['sku_id'],'o_id','f2_buytimes_')
###m2-m4
sample_sku2['f2_buytimes_in_7days_'] = sample_sku2['f2_buytimes_o_id_sku_id_nunique']/12
sample_sku2['f2_buytimes_in_15days_'] = sample_sku2['f2_buytimes_o_id_sku_id_nunique']/6
sample_sku2['f2_buytimes_in_31days_'] = sample_sku2['f2_buytimes_o_id_sku_id_nunique']/3

###m5
sample_sku2 = feat_sum(sample_sku2,sku2,['sku_id'],'o_sku_num','f2_buynums_')
###m6-m8
sample_sku2['f2_buynums_in_7days_'] = sample_sku2['f2_buynums_o_sku_num_sku_id_sum']/12
sample_sku2['f2_buynums_in_15days_'] = sample_sku2['f2_buynums_o_sku_num_sku_id_sum']/6
sample_sku2['f2_buynums_in_31days_'] = sample_sku2['f2_buynums_o_sku_num_sku_id_sum']/3

###m9
sample_sku2 = feat_sum(sample_sku2,sku2[sku2.a_type==1],['sku_id'],'a_num','f2_scannums_')
###m10-m12
sample_sku2['f2_scannums_in_7days_'] = sample_sku2['f2_scannums_a_num_sku_id_sum']/12
sample_sku2['f2_scannums_in_15days_'] = sample_sku2['f2_scannums_a_num_sku_id_sum']/6
sample_sku2['f2_scannums_in_31days_'] = sample_sku2['f2_scannums_a_num_sku_id_sum']/3

###m13
sample_sku2 = feat_sum(sample_sku2,sku2[sku2.a_type==2],['sku_id'],'a_num','f2_attnums_')
###m14-m16
sample_sku2['f2_attnums_in_7days_'] = sample_sku2['f2_attnums_a_num_sku_id_sum']/12
sample_sku2['f2_attnums_in_15days_'] = sample_sku2['f2_attnums_a_num_sku_id_sum']/6
sample_sku2['f2_attnums_in_31days_'] = sample_sku2['f2_attnums_a_num_sku_id_sum']/3

###m17
sample_sku2 = feat_nunique(sample_sku2,sku2,['sku_id'],'comment_create_tm','f2_commentnums_')
###m18-20
sample_sku2['f2_commentnums_in_7days_'] = sample_sku2['f2_commentnums_comment_create_tm_sku_id_nunique']/12
sample_sku2['f2_commentnums_in_15days_'] = sample_sku2['f2_commentnums_comment_create_tm_sku_id_nunique']/6
sample_sku2['f2_commentnums_in_31days_'] = sample_sku2['f2_commentnums_comment_create_tm_sku_id_nunique']/3

###m21
t1 = sample_sku2[['sku_id','f2_buytimes_o_id_sku_id_nunique']]
t2 = sku2.groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku2.groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f2_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f2_buytimes_o_id_sku_id_nunique']
t = t[['sku_id','f2_average_buy_day_gap']]
sample_sku2 = sample_sku2.merge(t,on=['sku_id'],how='left')

###m22
t1 = sample_sku2[['sku_id','f2_scannums_a_num_sku_id_sum']]
t2 = sku2[sku2.a_type==1].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku2[sku2.a_type==1].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f2_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f2_scannums_a_num_sku_id_sum']
t = t[['sku_id','f2_average_scan_day_gap']]
sample_sku2 = sample_sku2.merge(t,on=['sku_id'],how='left')
sample_sku2

###m23
t1 = sample_sku2[['sku_id','f2_attnums_a_num_sku_id_sum']]
t2 = sku2[sku2.a_type==2].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku2[sku2.a_type==2].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f2_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f2_attnums_a_num_sku_id_sum']
t = t[['sku_id','f2_average_att_day_gap']]
sample_sku2 = sample_sku2.merge(t,on=['sku_id'],how='left')


###m24
t1 = sample_sku2[['sku_id','f2_commentnums_comment_create_tm_sku_id_nunique']]
t2 = sku2.groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku2.groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f2_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f2_commentnums_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f2_average_comment_day_gap']]
sample_sku2 = sample_sku2.merge(t,on=['sku_id'],how='left')


###m25-m32还没写，正在考虑

sku2_a_o_date = sku2[((pd.to_datetime(sku2.a_date)<=pd.to_datetime(sku2.o_date.fillna('2018-01-01'))))]
sku2_a_o_date = sku2_a_o_date.sort_values(by=['sku_id','a_date','o_date']).drop_duplicates(['sku_id','a_date'],keep="first")
sku2_a_o_date = sku2_a_o_date[['sku_id','a_date','o_date','cate']]
sku2_a_o_date['o_a_gap']=(pd.to_datetime(sku2_a_o_date.o_date)-pd.to_datetime(sku2_a_o_date.a_date)).dt.days 
FLAG='sku2_a_o_'
sample_sku2 = feat_count(sample_sku2,sku2_a_o_date[sku2_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG+'order_')
sample_sku2 = feat_mean(sample_sku2,sku2_a_o_date[sku2_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG)

###m33
t1 = sample_sku2[['sku_id','f2_buynums_o_sku_num_sku_id_sum']] 
t2 = sku2[['sku_id','price']]
t3 = t1.merge(t2,on=['sku_id'],how='left')
t3['f2_total_price'] = t3['f2_buynums_o_sku_num_sku_id_sum']*t3['price']
t3 = t3[['sku_id','f2_total_price']]
sample_sku2 = sample_sku2.merge(t3,on=['sku_id'],how='left')

###m34-m36
sample_sku2['f2_total_price_in_7days_'] = sample_sku2['f2_total_price']/12
sample_sku2['f2_total_price_in_15days_'] = sample_sku2['f2_total_price']/6
sample_sku2['f2_total_price_in_31days_'] = sample_sku2['f2_total_price']/3

###m27-m40
sample_sku2 = feat_min(sample_sku2,sku2,['sku_id'],'o_day','f2_buytime_early_')
sample_sku2 = feat_min(sample_sku2,sku2[sku2.a_type==1],['sku_id'],'a_day','f2_scantime_early_')
sample_sku2 = feat_min(sample_sku2,sku2[sku2.a_type==2],['sku_id'],'a_day','f2_atttime_early_')
sample_sku2 = feat_min(sample_sku2,sku2,['sku_id'],'comment_day','f2_commenttime_early_')

###m41-m44
sample_sku2 = feat_max(sample_sku2,sku2,['sku_id'],'o_day','f2_buytime_late_')
sample_sku2 = feat_max(sample_sku2,sku2[sku2.a_type==1],['sku_id'],'a_day','f2_scantime_late_')
sample_sku2 = feat_max(sample_sku2,sku2[sku2.a_type==2],['sku_id'],'a_day','f2_atttime_late_')
sample_sku2 = feat_max(sample_sku2,sku2,['sku_id'],'comment_day','f2_commenttime_late_')


sample_sku2 = sample_sku2.drop_duplicates()
gc.collect()

MemoryError: 

In [16]:
####商品特征
#######dataset1_sku
#######
####for dataset1
sku1 = feature1[['sku_id','o_id','o_date','o_sku_num','o_month','o_day','comment_create_tm','score_level','comment_year','comment_month','comment_day','price','cate','para_1','a_date','a_num','a_type','a_year','a_month','a_day']]
t = sku1[['sku_id','price','cate']]
t = sku1[(sku1.cate==30)|(sku1.cate==101)]
t.drop_duplicates(inplace=True)
###m1
sample_sku1 = feat_nunique(t,sku1,['sku_id'],'o_id','f1_buytimes_')
###m2-m4
sample_sku1['f1_buytimes_in_7days_'] = sample_sku1['f1_buytimes_o_id_sku_id_nunique']/12
sample_sku1['f1_buytimes_in_15days_'] = sample_sku1['f1_buytimes_o_id_sku_id_nunique']/6
sample_sku1['f1_buytimes_in_31days_'] = sample_sku1['f1_buytimes_o_id_sku_id_nunique']/3

###m5
sample_sku1 = feat_sum(sample_sku1,sku1,['sku_id'],'o_sku_num','f1_buynums_')
###m6-m8
sample_sku1['f1_buynums_in_7days_'] = sample_sku1['f1_buynums_o_sku_num_sku_id_sum']/12
sample_sku1['f1_buynums_in_15days_'] = sample_sku1['f1_buynums_o_sku_num_sku_id_sum']/6
sample_sku1['f1_buynums_in_31days_'] = sample_sku1['f1_buynums_o_sku_num_sku_id_sum']/3

###m9
sample_sku1 = feat_sum(sample_sku1,sku1[sku1.a_type==1],['sku_id'],'a_num','f1_scannums_')
###m10-m12
sample_sku1['f1_scannums_in_7days_'] = sample_sku1['f1_scannums_a_num_sku_id_sum']/12
sample_sku1['f1_scannums_in_15days_'] = sample_sku1['f1_scannums_a_num_sku_id_sum']/6
sample_sku1['f1_scannums_in_31days_'] = sample_sku1['f1_scannums_a_num_sku_id_sum']/3

###m13
sample_sku1 = feat_sum(sample_sku1,sku1[sku1.a_type==2],['sku_id'],'a_num','f1_attnums_')
###m14-m16
sample_sku1['f1_attnums_in_7days_'] = sample_sku1['f1_attnums_a_num_sku_id_sum']/12
sample_sku1['f1_attnums_in_15days_'] = sample_sku1['f1_attnums_a_num_sku_id_sum']/6
sample_sku1['f1_attnums_in_31days_'] = sample_sku1['f1_attnums_a_num_sku_id_sum']/3

###m17
sample_sku1 = feat_nunique(sample_sku1,sku1,['sku_id'],'comment_create_tm','f1_commentnums_')
###m18-20
sample_sku1['f1_commentnums_in_7days_'] = sample_sku1['f1_commentnums_comment_create_tm_sku_id_nunique']/12
sample_sku1['f1_commentnums_in_15days_'] = sample_sku1['f1_commentnums_comment_create_tm_sku_id_nunique']/6
sample_sku1['f1_commentnums_in_31days_'] = sample_sku1['f1_commentnums_comment_create_tm_sku_id_nunique']/3

###m21
t1 = sample_sku1[['sku_id','f1_buytimes_o_id_sku_id_nunique']]
t2 = sku1.groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku1.groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f1_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f1_buytimes_o_id_sku_id_nunique']
t = t[['sku_id','f1_average_buy_day_gap']]
sample_sku1 = sample_sku1.merge(t,on=['sku_id'],how='left')

###m22
t1 = sample_sku1[['sku_id','f1_scannums_a_num_sku_id_sum']]
t2 = sku1[sku1.a_type==1].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku1[sku1.a_type==1].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f1_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f1_scannums_a_num_sku_id_sum']
t = t[['sku_id','f1_average_scan_day_gap']]
sample_sku1 = sample_sku1.merge(t,on=['sku_id'],how='left')
sample_sku1

###m23
t1 = sample_sku1[['sku_id','f1_attnums_a_num_sku_id_sum']]
t2 = sku1[sku1.a_type==2].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku1[sku1.a_type==2].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f1_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f1_attnums_a_num_sku_id_sum']
t = t[['sku_id','f1_average_att_day_gap']]
sample_sku1 = sample_sku1.merge(t,on=['sku_id'],how='left')


###m24
t1 = sample_sku1[['sku_id','f1_commentnums_comment_create_tm_sku_id_nunique']]
t2 = sku1.groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = sku1.groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f1_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f1_commentnums_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f1_average_comment_day_gap']]
sample_sku1 = sample_sku1.merge(t,on=['sku_id'],how='left')


###m25-m32还没写，正在考虑
sku1_a_o_date = sku1[((pd.to_datetime(sku1.a_date)<=pd.to_datetime(sku1.o_date.fillna('2018-01-01'))))]
sku1_a_o_date = sku1_a_o_date.sort_values(by=['sku_id','a_date','o_date']).drop_duplicates(['sku_id','a_date'],keep="first")
sku1_a_o_date = sku1_a_o_date[['sku_id','a_date','o_date','cate']]
sku1_a_o_date['o_a_gap']=(pd.to_datetime(sku1_a_o_date.o_date)-pd.to_datetime(sku1_a_o_date.a_date)).dt.days 
FLAG='sku1_a_o_'
sample_sku1 = feat_count(sample_sku1,sku1_a_o_date[sku1_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG+'order_')
sample_sku1 = feat_mean(sample_sku1,sku1_a_o_date[sku1_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG)

###m33
t1 = sample_sku1[['sku_id','f1_buynums_o_sku_num_sku_id_sum']] 
t2 = sku1[['sku_id','price']]
t3 = t1.merge(t2,on=['sku_id'],how='left')
t3['f1_total_price'] = t3['f1_buynums_o_sku_num_sku_id_sum']*t3['price']
t3 = t3[['sku_id','f1_total_price']]
sample_sku1 = sample_sku1.merge(t3,on=['sku_id'],how='left')

###m34-m36
sample_sku1['f1_total_price_in_7days_'] = sample_sku1['f1_total_price']/12
sample_sku1['f1_total_price_in_15days_'] = sample_sku1['f1_total_price']/6
sample_sku1['f1_total_price_in_31days_'] = sample_sku1['f1_total_price']/3

###m17-m40
sample_sku1 = feat_min(sample_sku1,sku1,['sku_id'],'o_day','f1_buytime_early_')
sample_sku1 = feat_min(sample_sku1,sku1[sku1.a_type==1],['sku_id'],'a_day','f1_scantime_early_')
sample_sku1 = feat_min(sample_sku1,sku1[sku1.a_type==2],['sku_id'],'a_day','f1_atttime_early_')
sample_sku1 = feat_min(sample_sku1,sku1,['sku_id'],'comment_day','f1_commenttime_early_')

###m41-m44
sample_sku1 = feat_max(sample_sku1,sku1,['sku_id'],'o_day','f1_buytime_late_')
sample_sku1 = feat_max(sample_sku1,sku1[sku1.a_type==1],['sku_id'],'a_day','f1_scantime_late_')
sample_sku1 = feat_max(sample_sku1,sku1[sku1.a_type==2],['sku_id'],'a_day','f1_atttime_late_')
sample_sku1 = feat_max(sample_sku1,sku1,['sku_id'],'comment_day','f1_commenttime_late_')


sample_sku1 = sample_sku1.drop_duplicates()
gc.collect()

142

In [17]:
###用户特征
#######dataset3_user
#######
####for dataset3
user3 = feature3[['user_id','sku_id','price','o_id','o_date','o_sku_num','o_month','o_day','comment_create_tm','score_level','comment_year','comment_month','comment_day','age','user_lv_cd','a_date','a_num','a_type','a_year','a_month','a_day']]
t = user3[['user_id','age','user_lv_cd','cate']]
t.drop_duplicates(inplace=True)
###u1
sample_user3 = feat_nunique(t,user3,['user_id'],'o_id','f3_buytimes_')
###u2-u4
sample_user3['f3_buytimes_in_7days_'] = sample_user3['f3_buytimes_o_id_user_id_nunique']/12
sample_user3['f3_buytimes_in_15days_'] = sample_user3['f3_buytimes_o_id_user_id_nunique']/6
sample_user3['f3_buytimes_in_31days_'] = sample_user3['f3_buytimes_o_id_user_id_nunique']/3

###u5
sample_user3 = feat_sum(sample_user3,user3,['user_id'],'o_sku_num','f3_buynums_')
###u6-u8
sample_user3['f3_buynums_in_7days_'] = sample_user3['f3_buynums_o_sku_num_user_id_sum']/12
sample_user3['f3_buynums_in_15days_'] = sample_user3['f3_buynums_o_sku_num_user_id_sum']/6
sample_user3['f3_buynums_in_31days_'] = sample_user3['f3_buynums_o_sku_num_user_id_sum']/3

###u9
sample_user3 = feat_sum(sample_user3,user3[user3.a_type==1],['user_id'],'a_num','f3_scannums_')
###u10-u12
sample_user3['f3_scannums_in_7days_'] = sample_user3['f3_scannums_a_num_user_id_sum']/12
sample_user3['f3_scannums_in_15days_'] = sample_user3['f3_scannums_a_num_user_id_sum']/6
sample_user3['f3_scannums_in_31days_'] = sample_user3['f3_scannums_a_num_user_id_sum']/3

###u13
sample_user3 = feat_sum(sample_user3,user3[user3.a_type==2],['user_id'],'a_num','f3_attnums_')
###u14-u16
sample_user3['f3_attnums_in_7days_'] = sample_user3['f3_attnums_a_num_user_id_sum']/12
sample_user3['f3_attnums_in_15days_'] = sample_user3['f3_attnums_a_num_user_id_sum']/6
sample_user3['f3_attnums_in_31days_'] = sample_user3['f3_attnums_a_num_user_id_sum']/3

###u17
sample_user3 = feat_nunique(sample_user3,user3,['user_id'],'comment_create_tm','f3_commentnums_')
###u18-20
sample_user3['f3_commentnums_in_7days_'] = sample_user3['f3_commentnums_comment_create_tm_user_id_nunique']/12
sample_user3['f3_commentnums_in_15days_'] = sample_user3['f3_commentnums_comment_create_tm_user_id_nunique']/6
sample_user3['f3_commentnums_in_31days_'] = sample_user3['f3_commentnums_comment_create_tm_user_id_nunique']/3

###u21
t1 = sample_user3[['user_id','f3_buytimes_o_id_user_id_nunique']]
t2 = user3.groupby(['user_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['user_id'],how='left')
t3 = user3.groupby(['user_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['user_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_o_id_user_id_nunique']
t = t[['user_id','f3_average_buy_day_gap']]
sample_user3 = sample_user3.merge(t,on=['user_id'],how='left')

###u22
t1 = sample_user3[['user_id','f3_scannums_a_num_user_id_sum']]
t2 = user3[user3.a_type==1].groupby(['user_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['user_id'],how='left')
t3 = user3[user3.a_type==1].groupby(['user_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['user_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_a_num_user_id_sum']
t = t[['user_id','f3_average_scan_day_gap']]
sample_user3 = sample_user3.merge(t,on=['user_id'],how='left')


###u23
t1 = sample_user3[['user_id','f3_attnums_a_num_user_id_sum']]
t2 = user3[user3.a_type==2].groupby(['user_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['user_id'],how='left')
t3 = user3[user3.a_type==2].groupby(['user_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['user_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_a_num_user_id_sum']
t = t[['user_id','f3_average_att_day_gap']]
sample_user3 = sample_user3.merge(t,on=['user_id'],how='left')


###u24
t1 = sample_user3[['user_id','f3_commentnums_comment_create_tm_user_id_nunique']]
t2 = user3.groupby(['user_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['user_id'],how='left')
t3 = user3.groupby(['user_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['user_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_comment_create_tm_user_id_nunique']
t = t[['user_id','f3_average_comment_day_gap']]
sample_user3 = sample_user3.merge(t,on=['user_id'],how='left')


###u25-u32还没写，正在考虑
user3_a_o_date = user3[((pd.to_datetime(user3.a_date)<=pd.to_datetime(user3.o_date.fillna('2018-01-01'))))]
user3_a_o_date = user3_a_o_date.sort_values(by=['sku_id','a_date','o_date']).drop_duplicates(['sku_id','a_date'],keep="first")
user3_a_o_date = user3_a_o_date[['sku_id','a_date','o_date','cate']]
user3_a_o_date['o_a_gap']=(pd.to_datetime(user3_a_o_date.o_date)-pd.to_datetime(user3_a_o_date.a_date)).dt.days 
FLAG='user3_a_o_'
sample_user3 = feat_count(sample_user3,user3_a_o_date[user3_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG+'order_')
sample_user3 = feat_mean(sample_user3,user3_a_o_date[user3_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG)



###u33
t1 = user3[['user_id','o_sku_num','price']] 
t1['sku_total_price'] = t1['o_sku_num'] * t1['price']
sample_user3 = feat_sum(sample_user3,t1,['user_id'],'sku_total_price','f3_user_total_consumption_')

###u34-u36
sample_user3['f3_total_consumption_in_7days_'] = sample_user3['f3_user_total_consumption_sku_total_price_user_id_sum']/12
sample_user3['f3_total_consumption_in_15days_'] = sample_user3['f3_user_total_consumption_sku_total_price_user_id_sum']/6
sample_user3['f3_total_consumption_in_31days_'] = sample_user3['f3_user_total_consumption_sku_total_price_user_id_sum']/3


###u37-u40
sample_user3 = feat_min(sample_user3,user3,['user_id'],'o_day','f3_buytime_early_')
sample_user3 = feat_min(sample_user3,user3[user3.a_type==1],['user_id'],'a_day','f3_scantime_early_')
sample_user3 = feat_min(sample_user3,user3[user3.a_type==2],['user_id'],'a_day','f3_atttime_early_')
sample_user3 = feat_min(sample_user3,user3,['user_id'],'comment_day','f3_commenttime_early_')

###u41-u44
sample_user3 = feat_max(sample_user3,user3,['user_id'],'o_day','f3_buytime_late_')
sample_user3 = feat_max(sample_user3,user3[user3.a_type==1],['user_id'],'a_day','f3_scantime_late_')
sample_user3 = feat_max(sample_user3,user3[user3.a_type==2],['user_id'],'a_day','f3_atttime_late_')
sample_user3 = feat_max(sample_user3,user3,['user_id'],'comment_day','f3_commenttime_late_')


sample_user3 = sample_user3.drop_duplicates()
gc.collect()

133

In [ ]:
user3 = feature3[['user_id','sku_id','price','o_id','o_date','o_sku_num','o_month','o_day','comment_create_tm','score_level','comment_year','comment_month','comment_day','age','user_lv_cd','a_date','a_num','a_type','a_year','a_month','a_day']]
t = user3[['user_id','age','user_lv_cd','cate']]
t.drop_duplicates(inplace=True)
###m1
sample_user3 = feat_nunique(t,user3[(user3.cate==101)|(user3.cate==30)],['sku_id'],'o_id','f3_buytimes_30101_')
sample_user3 = feat_nunique(sample_user3,user3[(user3.cate==30)],['sku_id'],'o_id','f3_buytimes_30_')
sample_user3 = feat_nunique(sample_user3,user3[(user3.cate==101)],['sku_id'],'o_id','f3_buytimes_101_')
sample_user3 = feat_nunique(sample_user3,user3[(user3.cate!=101)&(user3.cate!=30)],['sku_id'],'o_id','f3_buytimes_no_30101_')

###m2-m4
sample_user3['f3_buytimes_30101_in_7days_'] = sample_user3['f3_buytimes_30101_o_id_sku_id_nunique']/12
sample_user3['f3_buytimes_30_in_7days_'] = sample_user3['f3_buytimes_30_o_id_sku_id_nunique']/12
sample_user3['f3_buytimes_101_in_7days_'] = sample_user3['f3_buytimes_101_o_id_sku_id_nunique']/12
sample_user3['f3_buytimes_no_30101_in_7days_'] = sample_user3['f3_buytimes_no_30101_o_id_sku_id_nunique']/12

sample_user3['f3_buytimes_30101_in_15days_'] = sample_user3['f3_buytimes_30101_o_id_sku_id_nunique']/6
sample_user3['f3_buytimes_30_in_15days_'] = sample_user3['f3_buytimes_30_o_id_sku_id_nunique']/6
sample_user3['f3_buytimes_101_in_15days_'] = sample_user3['f3_buytimes_101_o_id_sku_id_nunique']/6
sample_user3['f3_buytimes_no_30101_in_15days_'] = sample_user3['f3_buytimes_no_30101_o_id_sku_id_nunique']/6

sample_user3['f3_buytimes_30101_in_31days_'] = sample_user3['f3_buytimes_30101_o_id_sku_id_nunique']/3
sample_user3['f3_buytimes_30_in_31days_'] = sample_user3['f3_buytimes_30_o_id_sku_id_nunique']/3
sample_user3['f3_buytimes_101_in_31days_'] = sample_user3['f3_buytimes_101_o_id_sku_id_nunique']/3
sample_user3['f3_buytimes_no_30101_in_31days_'] = sample_user3['f3_buytimes_no_30101_o_id_sku_id_nunique']/3

###m5
sample_user3 = feat_sum(sample_user3,user3[(user3.cate==101)|(user3.cate==30)],['sku_id'],'o_sku_num','f3_buynums_30101_')
sample_user3 = feat_sum(sample_user3,user3[(user3.cate==30)],['sku_id'],'o_sku_num','f3_buynums_30_')
sample_user3 = feat_sum(sample_user3,user3[(user3.cate==101)],['sku_id'],'o_sku_num','f3_buynums_101_')
sample_user3 = feat_sum(sample_user3,user3[(user3.cate!=101)&(user3.cate!=30)],['sku_id'],'o_sku_num','f3_buynums_no_30101_')

###m6-m8
sample_user3['f3_buynums_30101_in_7days_'] = sample_user3['f3_buynums_30101_o_sku_num_sku_id_sum']/12
sample_user3['f3_buynums_30_in_7days_'] = sample_user3['f3_buynums_30_o_sku_num_sku_id_sum']/12
sample_user3['f3_buynums_101_in_7days_'] = sample_user3['f3_buynums_101_o_sku_num_sku_id_sum']/12
sample_user3['f3_buynums_no_30101_in_7days_'] = sample_user3['f3_buynums_no_30101_o_sku_num_sku_id_sum']/12

sample_user3['f3_buynums_30101_in_15days_'] = sample_user3['f3_buynums_30101_o_sku_num_sku_id_sum']/6
sample_user3['f3_buynums_30_in_15days_'] = sample_user3['f3_buynums_30_o_sku_num_sku_id_sum']/6
sample_user3['f3_buynums_101_in_15days_'] = sample_user3['f3_buynums_101_o_sku_num_sku_id_sum']/6
sample_user3['f3_buynums_no_30101_in_15days_'] = sample_user3['f3_buynums_no_30101_o_sku_num_sku_id_sum']/6

sample_user3['f3_buynums_30101_in_31days_'] = sample_user3['f3_buynums_30101_o_sku_num_sku_id_sum']/3
sample_user3['f3_buynums_30_in_31days_'] = sample_user3['f3_buynums_30_o_sku_num_sku_id_sum']/3
sample_user3['f3_buynums_101_in_31days_'] = sample_user3['f3_buynums_101_o_sku_num_sku_id_sum']/3
sample_user3['f3_buynums_no_30101_in_31days_'] = sample_user3['f3_buynums_no_30101_o_sku_num_sku_id_sum']/3

###m9
sample_user3 = feat_sum(sample_user3,user3[((user3.cate==101)|(user3.cate==30))&(user3.a_type==1)],['sku_id'],'a_num','f3_scannums_30101_')
sample_user3 = feat_sum(sample_user3,user3[((user3.cate==30))&(user3.a_type==1)],['sku_id'],'a_num','f3_scannums_30_')
sample_user3 = feat_sum(sample_user3,user3[((user3.cate==101))&(user3.a_type==1)],['sku_id'],'a_num','f3_scannums_101_')
sample_user3 = feat_sum(sample_user3,user3[((user3.cate!=101)&(user3.cate!=30))&(user3.a_type==1)],['sku_id'],'a_num','f3_scannums_no_30101_')
###m10-m12
sample_user3['f3_scannums_30101_in_7days_'] = sample_user3['f3_scannums_30101_a_num_sku_id_sum']/12
sample_user3['f3_scannums_30_in_7days_'] = sample_user3['f3_scannums_30_a_num_sku_id_sum']/12
sample_user3['f3_scannums_101_in_7days_'] = sample_user3['f3_scannums_101_a_num_sku_id_sum']/12
sample_user3['f3_scannums_no_30101_in_7days_'] = sample_user3['f3_scannums_no_30101_a_num_sku_id_sum']/12

sample_user3['f3_scannums_30101_in_15days_'] = sample_user3['f3_scannums_30101_a_num_sku_id_sum']/6
sample_user3['f3_scannums_30_in_15days_'] = sample_user3['f3_scannums_30_a_num_sku_id_sum']/6
sample_user3['f3_scannums_101_in_15days_'] = sample_user3['f3_scannums_101_a_num_sku_id_sum']/6
sample_user3['f3_scannums_no_30101_in_15days_'] = sample_user3['f3_scannums_no_30101_a_num_sku_id_sum']/6

sample_user3['f3_scannums_30101_in_31days_'] = sample_user3['f3_scannums_30101_a_num_sku_id_sum']/3
sample_user3['f3_scannums_30_in_31days_'] = sample_user3['f3_scannums_30_a_num_sku_id_sum']/3
sample_user3['f3_scannums_101_in_31days_'] = sample_user3['f3_scannums_101_a_num_sku_id_sum']/3
sample_user3['f3_scannums_no_30101_in_31days_'] = sample_user3['f3_scannums_no_30101_a_num_sku_id_sum']/3


###m13
sample_user3 = feat_sum(sample_user3,user3[((user3.cate==101)|(user3.cate==30))&(user3.a_type==2)],['sku_id'],'a_num','f3_attnums_30101_')
sample_user3 = feat_sum(sample_user3,user3[((user3.cate==30))&(user3.a_type==2)],['sku_id'],'a_num','f3_attnums_30_')
sample_user3 = feat_sum(sample_user3,user3[((user3.cate==101))&(user3.a_type==2)],['sku_id'],'a_num','f3_attnums_101_')
sample_user3 = feat_sum(sample_user3,user3[((user3.cate!=101)&(user3.cate!=30))&(user3.a_type==2)],['sku_id'],'a_num','f3_attnums_no_30101_')
###m14-m16
sample_user3['f3_attnums_30101_in_7days_'] = sample_user3['f3_attnums_30101_a_num_sku_id_sum']/12
sample_user3['f3_attnums_30_in_7days_'] = sample_user3['f3_attnums_30_a_num_sku_id_sum']/12
sample_user3['f3_attnums_101_in_7days_'] = sample_user3['f3_attnums_101_a_num_sku_id_sum']/12
sample_user3['f3_attnums_no_30101_in_7days_'] = sample_user3['f3_attnums_no_30101_a_num_sku_id_sum']/12

sample_user3['f3_attnums_30101_in_15days_'] = sample_user3['f3_attnums_30101_a_num_sku_id_sum']/6
sample_user3['f3_attnums_30_in_15days_'] = sample_user3['f3_attnums_30_a_num_sku_id_sum']/6
sample_user3['f3_attnums_101_in_15days_'] = sample_user3['f3_attnums_101_a_num_sku_id_sum']/6
sample_user3['f3_attnums_no_30101_in_15days_'] = sample_user3['f3_attnums_no_30101_a_num_sku_id_sum']/6

sample_user3['f3_attnums_30101_in_31days_'] = sample_user3['f3_attnums_30101_a_num_sku_id_sum']/3
sample_user3['f3_attnums_30_in_31days_'] = sample_user3['f3_attnums_30_a_num_sku_id_sum']/3
sample_user3['f3_attnums_101_in_31days_'] = sample_user3['f3_attnums_101_a_num_sku_id_sum']/3
sample_user3['f3_attnums_no_30101_in_31days_'] = sample_user3['f3_attnums_no_30101_a_num_sku_id_sum']/3



###m17
sample_user3 = feat_nunique(sample_user3,user3[(user3.cate==101)|(user3.cate==30)],['sku_id'],'comment_create_tm','f3_commentnums_30101_')
sample_user3 = feat_nunique(sample_user3,user3[(user3.cate==30)],['sku_id'],'comment_create_tm','f3_commentnums_30_')
sample_user3 = feat_nunique(sample_user3,user3[(user3.cate==101)],['sku_id'],'comment_create_tm','f3_commentnums_101_')
sample_user3 = feat_nunique(sample_user3,user3[(user3.cate!=101)&(user3.cate!=30)],['sku_id'],'comment_create_tm','f3_commentnums_no_30101_')

###m18-20
sample_user3['f3_commentnums_30101_in_7days_'] = sample_user3['f3_commentnums_30101_comment_create_tm_sku_id_nunique']/12
sample_user3['f3_commentnums_30_in_7days_'] = sample_user3['f3_commentnums_30_comment_create_tm_sku_id_nunique']/12
sample_user3['f3_commentnums_101_in_7days_'] = sample_user3['f3_commentnums_101_comment_create_tm_sku_id_nunique']/12
sample_user3['f3_commentnums_no_30101_in_7days_'] = sample_user3['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']/12

sample_user3['f3_commentnums_30101_in_15days_'] = sample_user3['f3_commentnums_30101_comment_create_tm_sku_id_nunique']/6
sample_user3['f3_commentnums_30_in_15days_'] = sample_user3['f3_commentnums_30_comment_create_tm_sku_id_nunique']/6
sample_user3['f3_commentnums_101_in_15days_'] = sample_user3['f3_commentnums_101_comment_create_tm_sku_id_nunique']/6
sample_user3['f3_commentnums_no_30101_in_15days_'] = sample_user3['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']/6

sample_user3['f3_commentnums_30101_in_31days_'] = sample_user3['f3_commentnums_30101_comment_create_tm_sku_id_nunique']/3
sample_user3['f3_commentnums_30_in_31days_'] = sample_user3['f3_commentnums_30_comment_create_tm_sku_id_nunique']/3
sample_user3['f3_commentnums_101_in_31days_'] = sample_user3['f3_commentnums_101_comment_create_tm_sku_id_nunique']/3
sample_user3['f3_commentnums_no_30101_in_31days_'] = sample_user3['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']/3

###m21
t1 = sample_user3[['sku_id','f3_buytimes_30101_o_id_sku_id_nunique']]
t2 = user3[(user3.cate==101)|(user3.cate==30)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.cate==101)|(user3.cate==30)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_30101_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_30101_buy_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user3[['sku_id','f3_buytimes_30_o_id_sku_id_nunique']]
t2 = user3[(user3.cate==30)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.cate==30)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_30_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_30_buy_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user3[['sku_id','f3_buytimes_101_o_id_sku_id_nunique']]
t2 = user3[(user3.cate==101)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.cate==101)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_101_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_101_buy_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user3[['sku_id','f3_buytimes_no_30101_o_id_sku_id_nunique']]
t2 = user3[(user3.cate!=101)&(user3.cate!=30)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.cate!=101)&(user3.cate!=30)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_no_30101_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_no_30101_buy_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')

###m22
t1 = sample_user3[['sku_id','f3_scannums_30101_a_num_sku_id_sum']]
t2 = user3[(user3.a_type==1)&((user3.cate==101)|(user3.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.a_type==1)&((user3.cate==101)|(user3.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30101_scan_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user3[['sku_id','f3_scannums_30_a_num_sku_id_sum']]
t2 = user3[(user3.a_type==1)&((user3.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.a_type==1)&((user3.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_30_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30_scan_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user3[['sku_id','f3_scannums_101_a_num_sku_id_sum']]
t2 = user3[(user3.a_type==1)&((user3.cate==101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.a_type==1)&((user3.cate==101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_101_scan_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user3[['sku_id','f3_scannums_no_30101_a_num_sku_id_sum']]
t2 = user3[(user3.a_type==1)&((user3.cate!=30)&(user3.cate!=101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.a_type==1)&((user3.cate!=30)&(user3.cate!=101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_no_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_no_30101_scan_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')


###m23
t1 = sample_user3[['sku_id','f3_attnums_30101_a_num_sku_id_sum']]
t2 = user3[(user3.a_type==2)&((user3.cate==101)|(user3.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.a_type==2)&((user3.cate==101)|(user3.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30101_att_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user3[['sku_id','f3_attnums_30_a_num_sku_id_sum']]
t2 = user3[(user3.a_type==2)&((user3.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.a_type==2)&((user3.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_30_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30_att_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user3[['sku_id','f3_attnums_101_a_num_sku_id_sum']]
t2 = user3[(user3.a_type==2)&((user3.cate==101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.a_type==2)&((user3.cate==101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_101_att_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user3[['sku_id','f3_attnums_no_30101_a_num_sku_id_sum']]
t2 = user3[(user3.a_type==2)&((user3.cate!=30)&(user3.cate!=101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.a_type==2)&((user3.cate!=30)&(user3.cate!=101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_no_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_no_30101_att_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')


###m24
t1 = sample_user3[['sku_id','f3_commentnums_30101_comment_create_tm_sku_id_nunique']]
t2 = user3[(user3.cate==101)|(user3.cate==30)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.cate==101)|(user3.cate==30)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_30101_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_30101_comment_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')

t1 = sample_user3[['sku_id','f3_commentnums_30_comment_create_tm_sku_id_nunique']]
t2 = user3[(user3.cate==30)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.cate==30)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_30_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_30_comment_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')


t1 = sample_user3[['sku_id','f3_commentnums_101_comment_create_tm_sku_id_nunique']]
t2 = user3[(user3.cate==101)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.cate==101)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_101_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_101_comment_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')


t1 = sample_user3[['sku_id','f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']]
t2 = user3[(user3.cate!=101)&(user3.cate!=30)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user3[(user3.cate!=101)&(user3.cate!=30)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_no_30101_comment_day_gap']]
sample_user3 = sample_user3.merge(t,on=['sku_id'],how='left')

###m25-m32还没写，正在考虑
user3_a_o_date = user3[((pd.to_datetime(user3.a_date)<=pd.to_datetime(user3.o_date.fillna('2018-01-01'))))]
user3_a_o_date = user3_a_o_date.sort_values(by=['sku_id','a_date','o_date']).drop_duplicates(['sku_id','a_date'],keep="first")
user3_a_o_date = user3_a_o_date[['sku_id','a_date','o_date','cate']]
user3_a_o_date['o_a_gap']=(pd.to_datetime(user3_a_o_date.o_date)-pd.to_datetime(user3_a_o_date.a_date)).dt.days 
FLAG='user3_a_o_'
sample_user3 = feat_count(sample_user3,user3_a_o_date[user3_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG+'order_')
sample_user3 = feat_mean(sample_user3,user3_a_o_date[(user3_a_o_date.o_a_gap>=-1)&((user3_a_o_date.cate==101)|(user3_a_o_date.cate==30))],['sku_id'],'o_a_gap',FLAG+'30101_')
sample_user3 = feat_mean(sample_user3,user3_a_o_date[(user3_a_o_date.o_a_gap>=-1)&((user3_a_o_date.cate==30))],['sku_id'],'o_a_gap',FLAG+'30_')
sample_user3 = feat_mean(sample_user3,user3_a_o_date[(user3_a_o_date.o_a_gap>=-1)&((user3_a_o_date.cate==101))],['sku_id'],'o_a_gap',FLAG+'101_')
sample_user3 = feat_mean(sample_user3,user3_a_o_date[(user3_a_o_date.o_a_gap>=-1)&((user3_a_o_date.cate!=101)&(user3_a_o_date.cate!=30))],['sku_id'],'o_a_gap',FLAG+'no_30101_')


###u33
t1 = user3[['user_id','o_sku_num','price','cate']] 
t1['sku_total_price'] = t1['o_sku_num'] * t1['price']
sample_user3 = feat_sum(sample_user3,t1[(t1.cate==30)|(t1.cate==101)],['user_id'],'sku_total_price','f3_user_total_consumption_30101_')
sample_user3 = feat_sum(sample_user3,t1[(t1.cate==30)],['user_id'],'sku_total_price','f3_user_total_consumption_30_')
sample_user3 = feat_sum(sample_user3,t1[(t1.cate==101)],['user_id'],'sku_total_price','f3_user_total_consumption_101_')
sample_user3 = feat_sum(sample_user3,t1[(t1.cate!=30)&(t1.cate!=101)],['user_id'],'sku_total_price','f3_user_total_consumption_no_30101_')

###u34-u36
sample_user3['f3_total_consumption_30101_in_7days_'] = sample_user3['f3_user_total_consumption_30101_sku_total_price_user_id_sum']/12
sample_user3['f3_total_consumption_30_in_7days_'] = sample_user3['f3_user_total_consumption_30_sku_total_price_user_id_sum']/12
sample_user3['f3_total_consumption_101_in_7days_'] = sample_user3['f3_user_total_consumption_101_sku_total_price_user_id_sum']/12
sample_user3['f3_total_consumption_no_30101_in_7days_'] = sample_user3['f3_user_total_consumption_no_30101_sku_total_price_user_id_sum']/12

sample_user3['f3_total_consumption_30101_in_15days_'] = sample_user3['f3_user_total_consumption_30101_sku_total_price_user_id_sum']/6
sample_user3['f3_total_consumption_30_in_15days_'] = sample_user3['f3_user_total_consumption_30_sku_total_price_user_id_sum']/6
sample_user3['f3_total_consumption_101_in_15days_'] = sample_user3['f3_user_total_consumption_101_sku_total_price_user_id_sum']/6
sample_user3['f3_total_consumption_no_30101_in_15days_'] = sample_user3['f3_user_total_consumption_no_30101_sku_total_price_user_id_sum']/6

sample_user3['f3_total_consumption_30101_in_31days_'] = sample_user3['f3_user_total_consumption_30101_sku_total_price_user_id_sum']/3
sample_user3['f3_total_consumption_30_in_31days_'] = sample_user3['f3_user_total_consumption_30_sku_total_price_user_id_sum']/3
sample_user3['f3_total_consumption_101_in_31days_'] = sample_user3['f3_user_total_consumption_101_sku_total_price_user_id_sum']/3
sample_user3['f3_total_consumption_no_30101_in_31days_'] = sample_user3['f3_user_total_consumption_no_30101_sku_total_price_user_id_sum']/3


###u37-u40
sample_user3 = feat_min(sample_user3,user3[()],['user_id'],'o_day','f3_buytime_early_')
sample_user3 = feat_min(sample_user3,user3[user3.a_type==1],['user_id'],'a_day','f3_scantime_early_')
sample_user3 = feat_min(sample_user3,user3[user3.a_type==2],['user_id'],'a_day','f3_atttime_early_')
sample_user3 = feat_min(sample_user3,user3,['user_id'],'comment_day','f3_commenttime_early_')

sample_user3 = feat_min(sample_user3,user3[(user3.cate==101)|(user3.cate==30)],['user_id'],'o_day','f3_buytime_early_30101_')
sample_user3 = feat_min(sample_user3,user3[(user3.cate==30)],['user_id'],'o_day','f3_buytime_early_30_')
sample_user3 = feat_min(sample_user3,user3[(user3.cate==101)],['user_id'],'o_day','f3_buytime_early_101_')
sample_user3 = feat_min(sample_user3,user3[(user3.cate!=101)&(user3.cate!=30)],['user_id'],'o_day','f3_buytime_early_no_30101_')

sample_user3 = feat_min(sample_user3,user3[((user3.cate==101)|(user3.cate==30))&(user3.a_type==1)],['user_id'],'a_day','f3_scantime_early_30101_')
sample_user3 = feat_min(sample_user3,user3[(user3.cate==30)&(user3.a_type==1)],['user_id'],'a_day','f3_scantime_early_30_')
sample_user3 = feat_min(sample_user3,user3[(user3.cate==101)&(user3.a_type==1)],['user_id'],'a_day','f3_scantime_early_101_')
sample_user3 = feat_min(sample_user3,user3[((user3.cate!=101)&(user3.cate!=30))&(user3.a_type==1)],['user_id'],'a_day','f3_scantime_early_30101_')

sample_user3 = feat_min(sample_user3,user3[((user3.cate==101)|(user3.cate==30))&(user3.a_type==2)],['user_id'],'a_day','f3_atttime_early_30101_')
sample_user3 = feat_min(sample_user3,user3[(user3.cate==30)&(user3.a_type==2)],['user_id'],'a_day','f3_atttime_early_30_')
sample_user3 = feat_min(sample_user3,user3[(user3.cate==101)&(user3.a_type==2)],['user_id'],'a_day','f3_atttime_early_101_')
sample_user3 = feat_min(sample_user3,user3[((user3.cate!=101)&(user3.cate!=30))&(user3.a_type==2)],['user_id'],'a_day','f3_atttime_early_no_30101_')

sample_user3 = feat_min(sample_user3,user3[(user3.cate==101)|(user3.cate==30)],['user_id'],'comment_day','f3_commenttime_early_30101_')
sample_user3 = feat_min(sample_user3,user3[(user3.cate==30)],['user_id'],'comment_day','f3_commenttime_early_30_')
sample_user3 = feat_min(sample_user3,user3[(user3.cate==101)],['user_id'],'comment_day','f3_commenttime_early_101_')
sample_user3 = feat_min(sample_user3,user3[(user3.cate!=101)&(user3.cate!=30)],['user_id'],'comment_day','f3_commenttime_early_no_30101_')


###u41-u44
sample_user3 = feat_max(sample_user3,user3[(user3.cate==101)|(user3.cate==30)],['user_id'],'o_day','f3_buytime_late_30101_')
sample_user3 = feat_max(sample_user3,user3[(user3.cate==30)],['user_id'],'o_day','f3_buytime_late_30_')
sample_user3 = feat_max(sample_user3,user3[(user3.cate==101)],['user_id'],'o_day','f3_buytime_late_101_')
sample_user3 = feat_max(sample_user3,user3[(user3.cate!=101)&(user3.cate!=30)],['user_id'],'o_day','f3_buytime_late_no_30101_')

sample_user3 = feat_max(sample_user3,user3[((user3.cate==101)|(user3.cate==30))&(user3.a_type==1)],['user_id'],'a_day','f3_scantime_late_30101_')
sample_user3 = feat_max(sample_user3,user3[(user3.cate==30)&(user3.a_type==1)],['user_id'],'a_day','f3_scantime_late_30_')
sample_user3 = feat_max(sample_user3,user3[(user3.cate==101)&(user3.a_type==1)],['user_id'],'a_day','f3_scantime_late_101_')
sample_user3 = feat_max(sample_user3,user3[((user3.cate!=101)&(user3.cate!=30))&(user3.a_type==1)],['user_id'],'a_day','f3_scantime_late_30101_')

sample_user3 = feat_max(sample_user3,user3[((user3.cate==101)|(user3.cate==30))&(user3.a_type==2)],['user_id'],'a_day','f3_atttime_late_30101_')
sample_user3 = feat_max(sample_user3,user3[(user3.cate==30)&(user3.a_type==2)],['user_id'],'a_day','f3_atttime_late_30_')
sample_user3 = feat_max(sample_user3,user3[(user3.cate==101)&(user3.a_type==2)],['user_id'],'a_day','f3_atttime_late_101_')
sample_user3 = feat_max(sample_user3,user3[((user3.cate!=101)&(user3.cate!=30))&(user3.a_type==2)],['user_id'],'a_day','f3_atttime_late_no_30101_')

sample_user3 = feat_max(sample_user3,user3[(user3.cate==101)|(user3.cate==30)],['user_id'],'comment_day','f3_commenttime_late_30101_')
sample_user3 = feat_max(sample_user3,user3[(user3.cate==30)],['user_id'],'comment_day','f3_commenttime_late_30_')
sample_user3 = feat_max(sample_user3,user3[(user3.cate==101)],['user_id'],'comment_day','f3_commenttime_late_101_')
sample_user3 = feat_max(sample_user3,user3[(user3.cate!=101)&(user3.cate!=30)],['user_id'],'comment_day','f3_commenttime_late_no_30101_')


sample_user3 = sample_user3.drop_duplicates()
gc.collect()

In [18]:
###用户特征
#######dataset2_user
#######
####for dataset2
user2 = feature2[['user_id','sku_id','price','o_id','o_date','o_sku_num','o_month','o_day','comment_create_tm','score_level','comment_year','comment_month','comment_day','age','user_lv_cd','a_date','a_num','a_type','a_year','a_month','a_day']]
t = user2[['user_id','age','user_lv_cd']]
t.drop_duplicates(inplace=True)
###u1
sample_user2 = feat_nunique(t,user2,['user_id'],'o_id','f2_buytimes_')
###u2-u4
sample_user2['f2_buytimes_in_7days_'] = sample_user2['f2_buytimes_o_id_user_id_nunique']/12
sample_user2['f2_buytimes_in_15days_'] = sample_user2['f2_buytimes_o_id_user_id_nunique']/6
sample_user2['f2_buytimes_in_31days_'] = sample_user2['f2_buytimes_o_id_user_id_nunique']/3

###u5
sample_user2 = feat_sum(sample_user2,user2,['user_id'],'o_sku_num','f2_buynums_')
###u6-u8
sample_user2['f2_buynums_in_7days_'] = sample_user2['f2_buynums_o_sku_num_user_id_sum']/12
sample_user2['f2_buynums_in_15days_'] = sample_user2['f2_buynums_o_sku_num_user_id_sum']/6
sample_user2['f2_buynums_in_31days_'] = sample_user2['f2_buynums_o_sku_num_user_id_sum']/3

###u9
sample_user2 = feat_sum(sample_user2,user2[user2.a_type==1],['user_id'],'a_num','f2_scannums_')
###u10-u12
sample_user2['f2_scannums_in_7days_'] = sample_user2['f2_scannums_a_num_user_id_sum']/12
sample_user2['f2_scannums_in_15days_'] = sample_user2['f2_scannums_a_num_user_id_sum']/6
sample_user2['f2_scannums_in_31days_'] = sample_user2['f2_scannums_a_num_user_id_sum']/3

###u13
sample_user2 = feat_sum(sample_user2,user2[user2.a_type==2],['user_id'],'a_num','f2_attnums_')
###u14-u16
sample_user2['f2_attnums_in_7days_'] = sample_user2['f2_attnums_a_num_user_id_sum']/12
sample_user2['f2_attnums_in_15days_'] = sample_user2['f2_attnums_a_num_user_id_sum']/6
sample_user2['f2_attnums_in_31days_'] = sample_user2['f2_attnums_a_num_user_id_sum']/3

###u17
sample_user2 = feat_nunique(sample_user2,user2,['user_id'],'comment_create_tm','f2_commentnums_')
###u18-20
sample_user2['f2_commentnums_in_7days_'] = sample_user2['f2_commentnums_comment_create_tm_user_id_nunique']/12
sample_user2['f2_commentnums_in_15days_'] = sample_user2['f2_commentnums_comment_create_tm_user_id_nunique']/6
sample_user2['f2_commentnums_in_31days_'] = sample_user2['f2_commentnums_comment_create_tm_user_id_nunique']/3

###u21
t1 = sample_user2[['user_id','f2_buytimes_o_id_user_id_nunique']]
t2 = user2.groupby(['user_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['user_id'],how='left')
t3 = user2.groupby(['user_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['user_id'],how='left')
t['f2_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f2_buytimes_o_id_user_id_nunique']
t = t[['user_id','f2_average_buy_day_gap']]
sample_user2 = sample_user2.merge(t,on=['user_id'],how='left')

###u22
t1 = sample_user2[['user_id','f2_scannums_a_num_user_id_sum']]
t2 = user2[user2.a_type==1].groupby(['user_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['user_id'],how='left')
t3 = user2[user2.a_type==1].groupby(['user_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['user_id'],how='left')
t['f2_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f2_scannums_a_num_user_id_sum']
t = t[['user_id','f2_average_scan_day_gap']]
sample_user2 = sample_user2.merge(t,on=['user_id'],how='left')


###u23
t1 = sample_user2[['user_id','f2_attnums_a_num_user_id_sum']]
t2 = user2[user2.a_type==2].groupby(['user_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['user_id'],how='left')
t3 = user2[user2.a_type==2].groupby(['user_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['user_id'],how='left')
t['f2_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f2_attnums_a_num_user_id_sum']
t = t[['user_id','f2_average_att_day_gap']]
sample_user2 = sample_user2.merge(t,on=['user_id'],how='left')


###u24
t1 = sample_user2[['user_id','f2_commentnums_comment_create_tm_user_id_nunique']]
t2 = user2.groupby(['user_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['user_id'],how='left')
t3 = user2.groupby(['user_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['user_id'],how='left')
t['f2_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f2_commentnums_comment_create_tm_user_id_nunique']
t = t[['user_id','f2_average_comment_day_gap']]
sample_user2 = sample_user2.merge(t,on=['user_id'],how='left')


###u25-u32还没写，正在考虑
user2_a_o_date = user2[((pd.to_datetime(user2.a_date)<=pd.to_datetime(user2.o_date.fillna('2018-01-01'))))]
user2_a_o_date = user2_a_o_date.sort_values(by=['sku_id','a_date','o_date']).drop_duplicates(['sku_id','a_date'],keep="first")
user2_a_o_date = user2_a_o_date[['sku_id','a_date','o_date','cate']]
user2_a_o_date['o_a_gap']=(pd.to_datetime(user2_a_o_date.o_date)-pd.to_datetime(user2_a_o_date.a_date)).dt.days 
FLAG='user2_a_o_'
sample_user2 = feat_count(sample_user2,user2_a_o_date[user2_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG+'order_')
sample_user2 = feat_mean(sample_user2,user2_a_o_date[user2_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG)

###u33
t1 = user2[['user_id','o_sku_num','price']] 
t1['sku_total_price'] = t1['o_sku_num'] * t1['price']
sample_user2 = feat_sum(sample_user2,t1,['user_id'],'sku_total_price','f2_user_total_consumption_')

###u34-u36
sample_user2['f2_total_consumption_in_7days_'] = sample_user2['f2_user_total_consumption_sku_total_price_user_id_sum']/12
sample_user2['f2_total_consumption_in_15days_'] = sample_user2['f2_user_total_consumption_sku_total_price_user_id_sum']/6
sample_user2['f2_total_consumption_in_31days_'] = sample_user2['f2_user_total_consumption_sku_total_price_user_id_sum']/3


###u37-u40
sample_user2 = feat_min(sample_user2,user2,['user_id'],'o_day','f2_buytime_early_')
sample_user2 = feat_min(sample_user2,user2[user2.a_type==1],['user_id'],'a_day','f2_scantime_early_')
sample_user2 = feat_min(sample_user2,user2[user2.a_type==2],['user_id'],'a_day','f2_atttime_early_')
sample_user2 = feat_min(sample_user2,user2,['user_id'],'comment_day','f2_commenttime_early_')

###u41-u44
sample_user2 = feat_max(sample_user2,user2,['user_id'],'o_day','f2_buytime_late_')
sample_user2 = feat_max(sample_user2,user2[user2.a_type==1],['user_id'],'a_day','f2_scantime_late_')
sample_user2 = feat_max(sample_user2,user2[user2.a_type==2],['user_id'],'a_day','f2_atttime_late_')
sample_user2 = feat_max(sample_user2,user2,['user_id'],'comment_day','f2_commenttime_late_')

sample_user2 = sample_user2.drop_duplicates()
gc.collect()


133

In [ ]:
user2 = feature3[['user_id','sku_id','price','o_id','o_date','o_sku_num','o_month','o_day','comment_create_tm','score_level','comment_year','comment_month','comment_day','age','user_lv_cd','a_date','a_num','a_type','a_year','a_month','a_day']]
t = user2[['user_id','age','user_lv_cd','cate']]
t.drop_duplicates(inplace=True)
###m1
sample_user2 = feat_nunique(t,user2[(user2.cate==101)|(user2.cate==30)],['sku_id'],'o_id','f3_buytimes_30101_')
sample_user2 = feat_nunique(sample_user2,user2[(user2.cate==30)],['sku_id'],'o_id','f3_buytimes_30_')
sample_user2 = feat_nunique(sample_user2,user2[(user2.cate==101)],['sku_id'],'o_id','f3_buytimes_101_')
sample_user2 = feat_nunique(sample_user2,user2[(user2.cate!=101)&(user2.cate!=30)],['sku_id'],'o_id','f3_buytimes_no_30101_')

###m2-m4
sample_user2['f3_buytimes_30101_in_7days_'] = sample_user2['f3_buytimes_30101_o_id_sku_id_nunique']/12
sample_user2['f3_buytimes_30_in_7days_'] = sample_user2['f3_buytimes_30_o_id_sku_id_nunique']/12
sample_user2['f3_buytimes_101_in_7days_'] = sample_user2['f3_buytimes_101_o_id_sku_id_nunique']/12
sample_user2['f3_buytimes_no_30101_in_7days_'] = sample_user2['f3_buytimes_no_30101_o_id_sku_id_nunique']/12

sample_user2['f3_buytimes_30101_in_15days_'] = sample_user2['f3_buytimes_30101_o_id_sku_id_nunique']/6
sample_user2['f3_buytimes_30_in_15days_'] = sample_user2['f3_buytimes_30_o_id_sku_id_nunique']/6
sample_user2['f3_buytimes_101_in_15days_'] = sample_user2['f3_buytimes_101_o_id_sku_id_nunique']/6
sample_user2['f3_buytimes_no_30101_in_15days_'] = sample_user2['f3_buytimes_no_30101_o_id_sku_id_nunique']/6

sample_user2['f3_buytimes_30101_in_31days_'] = sample_user2['f3_buytimes_30101_o_id_sku_id_nunique']/3
sample_user2['f3_buytimes_30_in_31days_'] = sample_user2['f3_buytimes_30_o_id_sku_id_nunique']/3
sample_user2['f3_buytimes_101_in_31days_'] = sample_user2['f3_buytimes_101_o_id_sku_id_nunique']/3
sample_user2['f3_buytimes_no_30101_in_31days_'] = sample_user2['f3_buytimes_no_30101_o_id_sku_id_nunique']/3

###m5
sample_user2 = feat_sum(sample_user2,user2[(user2.cate==101)|(user2.cate==30)],['sku_id'],'o_sku_num','f3_buynums_30101_')
sample_user2 = feat_sum(sample_user2,user2[(user2.cate==30)],['sku_id'],'o_sku_num','f3_buynums_30_')
sample_user2 = feat_sum(sample_user2,user2[(user2.cate==101)],['sku_id'],'o_sku_num','f3_buynums_101_')
sample_user2 = feat_sum(sample_user2,user2[(user2.cate!=101)&(user2.cate!=30)],['sku_id'],'o_sku_num','f3_buynums_no_30101_')

###m6-m8
sample_user2['f3_buynums_30101_in_7days_'] = sample_user2['f3_buynums_30101_o_sku_num_sku_id_sum']/12
sample_user2['f3_buynums_30_in_7days_'] = sample_user2['f3_buynums_30_o_sku_num_sku_id_sum']/12
sample_user2['f3_buynums_101_in_7days_'] = sample_user2['f3_buynums_101_o_sku_num_sku_id_sum']/12
sample_user2['f3_buynums_no_30101_in_7days_'] = sample_user2['f3_buynums_no_30101_o_sku_num_sku_id_sum']/12

sample_user2['f3_buynums_30101_in_15days_'] = sample_user2['f3_buynums_30101_o_sku_num_sku_id_sum']/6
sample_user2['f3_buynums_30_in_15days_'] = sample_user2['f3_buynums_30_o_sku_num_sku_id_sum']/6
sample_user2['f3_buynums_101_in_15days_'] = sample_user2['f3_buynums_101_o_sku_num_sku_id_sum']/6
sample_user2['f3_buynums_no_30101_in_15days_'] = sample_user2['f3_buynums_no_30101_o_sku_num_sku_id_sum']/6

sample_user2['f3_buynums_30101_in_31days_'] = sample_user2['f3_buynums_30101_o_sku_num_sku_id_sum']/3
sample_user2['f3_buynums_30_in_31days_'] = sample_user2['f3_buynums_30_o_sku_num_sku_id_sum']/3
sample_user2['f3_buynums_101_in_31days_'] = sample_user2['f3_buynums_101_o_sku_num_sku_id_sum']/3
sample_user2['f3_buynums_no_30101_in_31days_'] = sample_user2['f3_buynums_no_30101_o_sku_num_sku_id_sum']/3

###m9
sample_user2 = feat_sum(sample_user2,user2[((user2.cate==101)|(user2.cate==30))&(user2.a_type==1)],['sku_id'],'a_num','f3_scannums_30101_')
sample_user2 = feat_sum(sample_user2,user2[((user2.cate==30))&(user2.a_type==1)],['sku_id'],'a_num','f3_scannums_30_')
sample_user2 = feat_sum(sample_user2,user2[((user2.cate==101))&(user2.a_type==1)],['sku_id'],'a_num','f3_scannums_101_')
sample_user2 = feat_sum(sample_user2,user2[((user2.cate!=101)&(user2.cate!=30))&(user2.a_type==1)],['sku_id'],'a_num','f3_scannums_no_30101_')
###m10-m12
sample_user2['f3_scannums_30101_in_7days_'] = sample_user2['f3_scannums_30101_a_num_sku_id_sum']/12
sample_user2['f3_scannums_30_in_7days_'] = sample_user2['f3_scannums_30_a_num_sku_id_sum']/12
sample_user2['f3_scannums_101_in_7days_'] = sample_user2['f3_scannums_101_a_num_sku_id_sum']/12
sample_user2['f3_scannums_no_30101_in_7days_'] = sample_user2['f3_scannums_no_30101_a_num_sku_id_sum']/12

sample_user2['f3_scannums_30101_in_15days_'] = sample_user2['f3_scannums_30101_a_num_sku_id_sum']/6
sample_user2['f3_scannums_30_in_15days_'] = sample_user2['f3_scannums_30_a_num_sku_id_sum']/6
sample_user2['f3_scannums_101_in_15days_'] = sample_user2['f3_scannums_101_a_num_sku_id_sum']/6
sample_user2['f3_scannums_no_30101_in_15days_'] = sample_user2['f3_scannums_no_30101_a_num_sku_id_sum']/6

sample_user2['f3_scannums_30101_in_31days_'] = sample_user2['f3_scannums_30101_a_num_sku_id_sum']/3
sample_user2['f3_scannums_30_in_31days_'] = sample_user2['f3_scannums_30_a_num_sku_id_sum']/3
sample_user2['f3_scannums_101_in_31days_'] = sample_user2['f3_scannums_101_a_num_sku_id_sum']/3
sample_user2['f3_scannums_no_30101_in_31days_'] = sample_user2['f3_scannums_no_30101_a_num_sku_id_sum']/3


###m13
sample_user2 = feat_sum(sample_user2,user2[((user2.cate==101)|(user2.cate==30))&(user2.a_type==2)],['sku_id'],'a_num','f3_attnums_30101_')
sample_user2 = feat_sum(sample_user2,user2[((user2.cate==30))&(user2.a_type==2)],['sku_id'],'a_num','f3_attnums_30_')
sample_user2 = feat_sum(sample_user2,user2[((user2.cate==101))&(user2.a_type==2)],['sku_id'],'a_num','f3_attnums_101_')
sample_user2 = feat_sum(sample_user2,user2[((user2.cate!=101)&(user2.cate!=30))&(user2.a_type==2)],['sku_id'],'a_num','f3_attnums_no_30101_')
###m14-m16
sample_user2['f3_attnums_30101_in_7days_'] = sample_user2['f3_attnums_30101_a_num_sku_id_sum']/12
sample_user2['f3_attnums_30_in_7days_'] = sample_user2['f3_attnums_30_a_num_sku_id_sum']/12
sample_user2['f3_attnums_101_in_7days_'] = sample_user2['f3_attnums_101_a_num_sku_id_sum']/12
sample_user2['f3_attnums_no_30101_in_7days_'] = sample_user2['f3_attnums_no_30101_a_num_sku_id_sum']/12

sample_user2['f3_attnums_30101_in_15days_'] = sample_user2['f3_attnums_30101_a_num_sku_id_sum']/6
sample_user2['f3_attnums_30_in_15days_'] = sample_user2['f3_attnums_30_a_num_sku_id_sum']/6
sample_user2['f3_attnums_101_in_15days_'] = sample_user2['f3_attnums_101_a_num_sku_id_sum']/6
sample_user2['f3_attnums_no_30101_in_15days_'] = sample_user2['f3_attnums_no_30101_a_num_sku_id_sum']/6

sample_user2['f3_attnums_30101_in_31days_'] = sample_user2['f3_attnums_30101_a_num_sku_id_sum']/3
sample_user2['f3_attnums_30_in_31days_'] = sample_user2['f3_attnums_30_a_num_sku_id_sum']/3
sample_user2['f3_attnums_101_in_31days_'] = sample_user2['f3_attnums_101_a_num_sku_id_sum']/3
sample_user2['f3_attnums_no_30101_in_31days_'] = sample_user2['f3_attnums_no_30101_a_num_sku_id_sum']/3



###m17
sample_user2 = feat_nunique(sample_user2,user2[(user2.cate==101)|(user2.cate==30)],['sku_id'],'comment_create_tm','f3_commentnums_30101_')
sample_user2 = feat_nunique(sample_user2,user2[(user2.cate==30)],['sku_id'],'comment_create_tm','f3_commentnums_30_')
sample_user2 = feat_nunique(sample_user2,user2[(user2.cate==101)],['sku_id'],'comment_create_tm','f3_commentnums_101_')
sample_user2 = feat_nunique(sample_user2,user2[(user2.cate!=101)&(user2.cate!=30)],['sku_id'],'comment_create_tm','f3_commentnums_no_30101_')

###m18-20
sample_user2['f3_commentnums_30101_in_7days_'] = sample_user2['f3_commentnums_30101_comment_create_tm_sku_id_nunique']/12
sample_user2['f3_commentnums_30_in_7days_'] = sample_user2['f3_commentnums_30_comment_create_tm_sku_id_nunique']/12
sample_user2['f3_commentnums_101_in_7days_'] = sample_user2['f3_commentnums_101_comment_create_tm_sku_id_nunique']/12
sample_user2['f3_commentnums_no_30101_in_7days_'] = sample_user2['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']/12

sample_user2['f3_commentnums_30101_in_15days_'] = sample_user2['f3_commentnums_30101_comment_create_tm_sku_id_nunique']/6
sample_user2['f3_commentnums_30_in_15days_'] = sample_user2['f3_commentnums_30_comment_create_tm_sku_id_nunique']/6
sample_user2['f3_commentnums_101_in_15days_'] = sample_user2['f3_commentnums_101_comment_create_tm_sku_id_nunique']/6
sample_user2['f3_commentnums_no_30101_in_15days_'] = sample_user2['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']/6

sample_user2['f3_commentnums_30101_in_31days_'] = sample_user2['f3_commentnums_30101_comment_create_tm_sku_id_nunique']/3
sample_user2['f3_commentnums_30_in_31days_'] = sample_user2['f3_commentnums_30_comment_create_tm_sku_id_nunique']/3
sample_user2['f3_commentnums_101_in_31days_'] = sample_user2['f3_commentnums_101_comment_create_tm_sku_id_nunique']/3
sample_user2['f3_commentnums_no_30101_in_31days_'] = sample_user2['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']/3

###m21
t1 = sample_user2[['sku_id','f3_buytimes_30101_o_id_sku_id_nunique']]
t2 = user2[(user2.cate==101)|(user2.cate==30)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.cate==101)|(user2.cate==30)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_30101_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_30101_buy_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user2[['sku_id','f3_buytimes_30_o_id_sku_id_nunique']]
t2 = user2[(user2.cate==30)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.cate==30)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_30_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_30_buy_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user2[['sku_id','f3_buytimes_101_o_id_sku_id_nunique']]
t2 = user2[(user2.cate==101)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.cate==101)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_101_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_101_buy_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user2[['sku_id','f3_buytimes_no_30101_o_id_sku_id_nunique']]
t2 = user2[(user2.cate!=101)&(user2.cate!=30)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.cate!=101)&(user2.cate!=30)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_no_30101_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_no_30101_buy_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')

###m22
t1 = sample_user2[['sku_id','f3_scannums_30101_a_num_sku_id_sum']]
t2 = user2[(user2.a_type==1)&((user2.cate==101)|(user2.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.a_type==1)&((user2.cate==101)|(user2.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30101_scan_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user2[['sku_id','f3_scannums_30_a_num_sku_id_sum']]
t2 = user2[(user2.a_type==1)&((user2.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.a_type==1)&((user2.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_30_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30_scan_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user2[['sku_id','f3_scannums_101_a_num_sku_id_sum']]
t2 = user2[(user2.a_type==1)&((user2.cate==101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.a_type==1)&((user2.cate==101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_101_scan_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user2[['sku_id','f3_scannums_no_30101_a_num_sku_id_sum']]
t2 = user2[(user2.a_type==1)&((user2.cate!=30)&(user2.cate!=101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.a_type==1)&((user2.cate!=30)&(user2.cate!=101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_no_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_no_30101_scan_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')


###m23
t1 = sample_user2[['sku_id','f3_attnums_30101_a_num_sku_id_sum']]
t2 = user2[(user2.a_type==2)&((user2.cate==101)|(user2.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.a_type==2)&((user2.cate==101)|(user2.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30101_att_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user2[['sku_id','f3_attnums_30_a_num_sku_id_sum']]
t2 = user2[(user2.a_type==2)&((user2.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.a_type==2)&((user2.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_30_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30_att_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user2[['sku_id','f3_attnums_101_a_num_sku_id_sum']]
t2 = user2[(user2.a_type==2)&((user2.cate==101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.a_type==2)&((user2.cate==101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_101_att_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user2[['sku_id','f3_attnums_no_30101_a_num_sku_id_sum']]
t2 = user2[(user2.a_type==2)&((user2.cate!=30)&(user2.cate!=101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.a_type==2)&((user2.cate!=30)&(user2.cate!=101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_no_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_no_30101_att_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')


###m24
t1 = sample_user2[['sku_id','f3_commentnums_30101_comment_create_tm_sku_id_nunique']]
t2 = user2[(user2.cate==101)|(user2.cate==30)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.cate==101)|(user2.cate==30)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_30101_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_30101_comment_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')

t1 = sample_user2[['sku_id','f3_commentnums_30_comment_create_tm_sku_id_nunique']]
t2 = user2[(user2.cate==30)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.cate==30)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_30_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_30_comment_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')


t1 = sample_user2[['sku_id','f3_commentnums_101_comment_create_tm_sku_id_nunique']]
t2 = user2[(user2.cate==101)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.cate==101)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_101_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_101_comment_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')


t1 = sample_user2[['sku_id','f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']]
t2 = user2[(user2.cate!=101)&(user2.cate!=30)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user2[(user2.cate!=101)&(user2.cate!=30)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_no_30101_comment_day_gap']]
sample_user2 = sample_user2.merge(t,on=['sku_id'],how='left')

###m25-m32还没写，正在考虑
user2_a_o_date = user2[((pd.to_datetime(user2.a_date)<=pd.to_datetime(user2.o_date.fillna('2018-01-01'))))]
user2_a_o_date = user2_a_o_date.sort_values(by=['sku_id','a_date','o_date']).drop_duplicates(['sku_id','a_date'],keep="first")
user2_a_o_date = user2_a_o_date[['sku_id','a_date','o_date','cate']]
user2_a_o_date['o_a_gap']=(pd.to_datetime(user2_a_o_date.o_date)-pd.to_datetime(user2_a_o_date.a_date)).dt.days 
FLAG='user2_a_o_'
sample_user2 = feat_count(sample_user2,user2_a_o_date[user2_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG+'order_')
sample_user2 = feat_mean(sample_user2,user2_a_o_date[(user2_a_o_date.o_a_gap>=-1)&((user2_a_o_date.cate==101)|(user2_a_o_date.cate==30))],['sku_id'],'o_a_gap',FLAG+'30101_')
sample_user2 = feat_mean(sample_user2,user2_a_o_date[(user2_a_o_date.o_a_gap>=-1)&((user2_a_o_date.cate==30))],['sku_id'],'o_a_gap',FLAG+'30_')
sample_user2 = feat_mean(sample_user2,user2_a_o_date[(user2_a_o_date.o_a_gap>=-1)&((user2_a_o_date.cate==101))],['sku_id'],'o_a_gap',FLAG+'101_')
sample_user2 = feat_mean(sample_user2,user2_a_o_date[(user2_a_o_date.o_a_gap>=-1)&((user2_a_o_date.cate!=101)&(user2_a_o_date.cate!=30))],['sku_id'],'o_a_gap',FLAG+'no_30101_')


###u33
t1 = user2[['user_id','o_sku_num','price','cate']] 
t1['sku_total_price'] = t1['o_sku_num'] * t1['price']
sample_user2 = feat_sum(sample_user2,t1[(t1.cate==30)|(t1.cate==101)],['user_id'],'sku_total_price','f3_user_total_consumption_30101_')
sample_user2 = feat_sum(sample_user2,t1[(t1.cate==30)],['user_id'],'sku_total_price','f3_user_total_consumption_30_')
sample_user2 = feat_sum(sample_user2,t1[(t1.cate==101)],['user_id'],'sku_total_price','f3_user_total_consumption_101_')
sample_user2 = feat_sum(sample_user2,t1[(t1.cate!=30)&(t1.cate!=101)],['user_id'],'sku_total_price','f3_user_total_consumption_no_30101_')

###u34-u36
sample_user2['f3_total_consumption_30101_in_7days_'] = sample_user2['f3_user_total_consumption_30101_sku_total_price_user_id_sum']/12
sample_user2['f3_total_consumption_30_in_7days_'] = sample_user2['f3_user_total_consumption_30_sku_total_price_user_id_sum']/12
sample_user2['f3_total_consumption_101_in_7days_'] = sample_user2['f3_user_total_consumption_101_sku_total_price_user_id_sum']/12
sample_user2['f3_total_consumption_no_30101_in_7days_'] = sample_user2['f3_user_total_consumption_no_30101_sku_total_price_user_id_sum']/12

sample_user2['f3_total_consumption_30101_in_15days_'] = sample_user2['f3_user_total_consumption_30101_sku_total_price_user_id_sum']/6
sample_user2['f3_total_consumption_30_in_15days_'] = sample_user2['f3_user_total_consumption_30_sku_total_price_user_id_sum']/6
sample_user2['f3_total_consumption_101_in_15days_'] = sample_user2['f3_user_total_consumption_101_sku_total_price_user_id_sum']/6
sample_user2['f3_total_consumption_no_30101_in_15days_'] = sample_user2['f3_user_total_consumption_no_30101_sku_total_price_user_id_sum']/6

sample_user2['f3_total_consumption_30101_in_31days_'] = sample_user2['f3_user_total_consumption_30101_sku_total_price_user_id_sum']/3
sample_user2['f3_total_consumption_30_in_31days_'] = sample_user2['f3_user_total_consumption_30_sku_total_price_user_id_sum']/3
sample_user2['f3_total_consumption_101_in_31days_'] = sample_user2['f3_user_total_consumption_101_sku_total_price_user_id_sum']/3
sample_user2['f3_total_consumption_no_30101_in_31days_'] = sample_user2['f3_user_total_consumption_no_30101_sku_total_price_user_id_sum']/3


###u37-u40
sample_user2 = feat_min(sample_user2,user2[()],['user_id'],'o_day','f3_buytime_early_')
sample_user2 = feat_min(sample_user2,user2[user2.a_type==1],['user_id'],'a_day','f3_scantime_early_')
sample_user2 = feat_min(sample_user2,user2[user2.a_type==2],['user_id'],'a_day','f3_atttime_early_')
sample_user2 = feat_min(sample_user2,user2,['user_id'],'comment_day','f3_commenttime_early_')

sample_user2 = feat_min(sample_user2,user2[(user2.cate==101)|(user2.cate==30)],['user_id'],'o_day','f3_buytime_early_30101_')
sample_user2 = feat_min(sample_user2,user2[(user2.cate==30)],['user_id'],'o_day','f3_buytime_early_30_')
sample_user2 = feat_min(sample_user2,user2[(user2.cate==101)],['user_id'],'o_day','f3_buytime_early_101_')
sample_user2 = feat_min(sample_user2,user2[(user2.cate!=101)&(user2.cate!=30)],['user_id'],'o_day','f3_buytime_early_no_30101_')

sample_user2 = feat_min(sample_user2,user2[((user2.cate==101)|(user2.cate==30))&(user2.a_type==1)],['user_id'],'a_day','f3_scantime_early_30101_')
sample_user2 = feat_min(sample_user2,user2[(user2.cate==30)&(user2.a_type==1)],['user_id'],'a_day','f3_scantime_early_30_')
sample_user2 = feat_min(sample_user2,user2[(user2.cate==101)&(user2.a_type==1)],['user_id'],'a_day','f3_scantime_early_101_')
sample_user2 = feat_min(sample_user2,user2[((user2.cate!=101)&(user2.cate!=30))&(user2.a_type==1)],['user_id'],'a_day','f3_scantime_early_30101_')

sample_user2 = feat_min(sample_user2,user2[((user2.cate==101)|(user2.cate==30))&(user2.a_type==2)],['user_id'],'a_day','f3_atttime_early_30101_')
sample_user2 = feat_min(sample_user2,user2[(user2.cate==30)&(user2.a_type==2)],['user_id'],'a_day','f3_atttime_early_30_')
sample_user2 = feat_min(sample_user2,user2[(user2.cate==101)&(user2.a_type==2)],['user_id'],'a_day','f3_atttime_early_101_')
sample_user2 = feat_min(sample_user2,user2[((user2.cate!=101)&(user2.cate!=30))&(user2.a_type==2)],['user_id'],'a_day','f3_atttime_early_no_30101_')

sample_user2 = feat_min(sample_user2,user2[(user2.cate==101)|(user2.cate==30)],['user_id'],'comment_day','f3_commenttime_early_30101_')
sample_user2 = feat_min(sample_user2,user2[(user2.cate==30)],['user_id'],'comment_day','f3_commenttime_early_30_')
sample_user2 = feat_min(sample_user2,user2[(user2.cate==101)],['user_id'],'comment_day','f3_commenttime_early_101_')
sample_user2 = feat_min(sample_user2,user2[(user2.cate!=101)&(user2.cate!=30)],['user_id'],'comment_day','f3_commenttime_early_no_30101_')


###u41-u44
sample_user2 = feat_max(sample_user2,user2[(user2.cate==101)|(user2.cate==30)],['user_id'],'o_day','f3_buytime_late_30101_')
sample_user2 = feat_max(sample_user2,user2[(user2.cate==30)],['user_id'],'o_day','f3_buytime_late_30_')
sample_user2 = feat_max(sample_user2,user2[(user2.cate==101)],['user_id'],'o_day','f3_buytime_late_101_')
sample_user2 = feat_max(sample_user2,user2[(user2.cate!=101)&(user2.cate!=30)],['user_id'],'o_day','f3_buytime_late_no_30101_')

sample_user2 = feat_max(sample_user2,user2[((user2.cate==101)|(user2.cate==30))&(user2.a_type==1)],['user_id'],'a_day','f3_scantime_late_30101_')
sample_user2 = feat_max(sample_user2,user2[(user2.cate==30)&(user2.a_type==1)],['user_id'],'a_day','f3_scantime_late_30_')
sample_user2 = feat_max(sample_user2,user2[(user2.cate==101)&(user2.a_type==1)],['user_id'],'a_day','f3_scantime_late_101_')
sample_user2 = feat_max(sample_user2,user2[((user2.cate!=101)&(user2.cate!=30))&(user2.a_type==1)],['user_id'],'a_day','f3_scantime_late_30101_')

sample_user2 = feat_max(sample_user2,user2[((user2.cate==101)|(user2.cate==30))&(user2.a_type==2)],['user_id'],'a_day','f3_atttime_late_30101_')
sample_user2 = feat_max(sample_user2,user2[(user2.cate==30)&(user2.a_type==2)],['user_id'],'a_day','f3_atttime_late_30_')
sample_user2 = feat_max(sample_user2,user2[(user2.cate==101)&(user2.a_type==2)],['user_id'],'a_day','f3_atttime_late_101_')
sample_user2 = feat_max(sample_user2,user2[((user2.cate!=101)&(user2.cate!=30))&(user2.a_type==2)],['user_id'],'a_day','f3_atttime_late_no_30101_')

sample_user2 = feat_max(sample_user2,user2[(user2.cate==101)|(user2.cate==30)],['user_id'],'comment_day','f3_commenttime_late_30101_')
sample_user2 = feat_max(sample_user2,user2[(user2.cate==30)],['user_id'],'comment_day','f3_commenttime_late_30_')
sample_user2 = feat_max(sample_user2,user2[(user2.cate==101)],['user_id'],'comment_day','f3_commenttime_late_101_')
sample_user2 = feat_max(sample_user2,user2[(user2.cate!=101)&(user2.cate!=30)],['user_id'],'comment_day','f3_commenttime_late_no_30101_')


sample_user2 = sample_user2.drop_duplicates()
gc.collect()


In [19]:
###用户特征
#######dataset1_user
#######
####for dataset1
user1 = feature1[['user_id','sku_id','price','o_id','o_date','o_sku_num','o_month','o_day','comment_create_tm','score_level','comment_year','comment_month','comment_day','age','user_lv_cd','a_date','a_num','a_type','a_year','a_month','a_day']]
t = user1[['user_id','age','user_lv_cd']]
t.drop_duplicates(inplace=True)
###u1
sample_user1 = feat_nunique(t,user1,['user_id'],'o_id','f1_buytimes_')
###u2-u4
sample_user1['f1_buytimes_in_7days_'] = sample_user1['f1_buytimes_o_id_user_id_nunique']/12
sample_user1['f1_buytimes_in_15days_'] = sample_user1['f1_buytimes_o_id_user_id_nunique']/6
sample_user1['f1_buytimes_in_31days_'] = sample_user1['f1_buytimes_o_id_user_id_nunique']/3

###u5
sample_user1 = feat_sum(sample_user1,user1,['user_id'],'o_sku_num','f1_buynums_')
###u6-u8
sample_user1['f1_buynums_in_7days_'] = sample_user1['f1_buynums_o_sku_num_user_id_sum']/12
sample_user1['f1_buynums_in_15days_'] = sample_user1['f1_buynums_o_sku_num_user_id_sum']/6
sample_user1['f1_buynums_in_31days_'] = sample_user1['f1_buynums_o_sku_num_user_id_sum']/3

###u9
sample_user1 = feat_sum(sample_user1,user1[user1.a_type==1],['user_id'],'a_num','f1_scannums_')
###u10-u12
sample_user1['f1_scannums_in_7days_'] = sample_user1['f1_scannums_a_num_user_id_sum']/12
sample_user1['f1_scannums_in_15days_'] = sample_user1['f1_scannums_a_num_user_id_sum']/6
sample_user1['f1_scannums_in_31days_'] = sample_user1['f1_scannums_a_num_user_id_sum']/3

###u13
sample_user1 = feat_sum(sample_user1,user1[user1.a_type==2],['user_id'],'a_num','f1_attnums_')
###u14-u16
sample_user1['f1_attnums_in_7days_'] = sample_user1['f1_attnums_a_num_user_id_sum']/12
sample_user1['f1_attnums_in_15days_'] = sample_user1['f1_attnums_a_num_user_id_sum']/6
sample_user1['f1_attnums_in_31days_'] = sample_user1['f1_attnums_a_num_user_id_sum']/3

###u17
sample_user1 = feat_nunique(sample_user1,user1,['user_id'],'comment_create_tm','f1_commentnums_')
###u18-20
sample_user1['f1_commentnums_in_7days_'] = sample_user1['f1_commentnums_comment_create_tm_user_id_nunique']/12
sample_user1['f1_commentnums_in_15days_'] = sample_user1['f1_commentnums_comment_create_tm_user_id_nunique']/6
sample_user1['f1_commentnums_in_31days_'] = sample_user1['f1_commentnums_comment_create_tm_user_id_nunique']/3

###u21
t1 = sample_user1[['user_id','f1_buytimes_o_id_user_id_nunique']]
t2 = user1.groupby(['user_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['user_id'],how='left')
t3 = user1.groupby(['user_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['user_id'],how='left')
t['f1_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f1_buytimes_o_id_user_id_nunique']
t = t[['user_id','f1_average_buy_day_gap']]
sample_user1 = sample_user1.merge(t,on=['user_id'],how='left')

###u22
t1 = sample_user1[['user_id','f1_scannums_a_num_user_id_sum']]
t2 = user1[user1.a_type==1].groupby(['user_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['user_id'],how='left')
t3 = user1[user1.a_type==1].groupby(['user_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['user_id'],how='left')
t['f1_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f1_scannums_a_num_user_id_sum']
t = t[['user_id','f1_average_scan_day_gap']]
sample_user1 = sample_user1.merge(t,on=['user_id'],how='left')


###u23
t1 = sample_user1[['user_id','f1_attnums_a_num_user_id_sum']]
t2 = user1[user1.a_type==2].groupby(['user_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['user_id'],how='left')
t3 = user1[user1.a_type==2].groupby(['user_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['user_id'],how='left')
t['f1_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f1_attnums_a_num_user_id_sum']
t = t[['user_id','f1_average_att_day_gap']]
sample_user1 = sample_user1.merge(t,on=['user_id'],how='left')


###u24
t1 = sample_user1[['user_id','f1_commentnums_comment_create_tm_user_id_nunique']]
t2 = user1.groupby(['user_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['user_id'],how='left')
t3 = user1.groupby(['user_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['user_id'],how='left')
t['f1_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f1_commentnums_comment_create_tm_user_id_nunique']
t = t[['user_id','f1_average_comment_day_gap']]
sample_user1 = sample_user1.merge(t,on=['user_id'],how='left')


###u25-u32还没写，正在考虑
user1_a_o_date = user1[((pd.to_datetime(user1.a_date)<=pd.to_datetime(user1.o_date.fillna('2018-01-01'))))]
user1_a_o_date = user1_a_o_date.sort_values(by=['sku_id','a_date','o_date']).drop_duplicates(['sku_id','a_date'],keep="first")
user1_a_o_date = user1_a_o_date[['sku_id','a_date','o_date','cate']]
user1_a_o_date['o_a_gap']=(pd.to_datetime(user1_a_o_date.o_date)-pd.to_datetime(user1_a_o_date.a_date)).dt.days 
FLAG='user1_a_o_'
sample_user1 = feat_count(sample_user1,user1_a_o_date[user1_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG+'order_')
sample_user1 = feat_mean(sample_user1,user1_a_o_date[user1_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG)

###u33
t1 = user1[['user_id','o_sku_num','price']] 
t1['sku_total_price'] = t1['o_sku_num'] * t1['price']
sample_user1 = feat_sum(sample_user1,t1,['user_id'],'sku_total_price','f1_user_total_consumption_')

###u34-u36
sample_user1['f1_total_consumption_in_7days_'] = sample_user1['f1_user_total_consumption_sku_total_price_user_id_sum']/12
sample_user1['f1_total_consumption_in_15days_'] = sample_user1['f1_user_total_consumption_sku_total_price_user_id_sum']/6
sample_user1['f1_total_consumption_in_31days_'] = sample_user1['f1_user_total_consumption_sku_total_price_user_id_sum']/3

###u37-u40
sample_user1 = feat_min(sample_user1,user1,['user_id'],'o_day','f1_buytime_early_')
sample_user1 = feat_min(sample_user1,user1[user1.a_type==1],['user_id'],'a_day','f1_scantime_early_')
sample_user1 = feat_min(sample_user1,user1[user1.a_type==2],['user_id'],'a_day','f1_atttime_early_')
sample_user1 = feat_min(sample_user1,user1,['user_id'],'comment_day','f1_commenttime_early_')

###u41-u44
sample_user1 = feat_max(sample_user1,user1,['user_id'],'o_day','f1_buytime_late_')
sample_user1 = feat_max(sample_user1,user1[user1.a_type==1],['user_id'],'a_day','f1_scantime_late_')
sample_user1 = feat_max(sample_user1,user1[user1.a_type==2],['user_id'],'a_day','f1_atttime_late_')
sample_user1 = feat_max(sample_user1,user1,['user_id'],'comment_day','f1_commenttime_late_')

sample_user1 = sample_user1.drop_duplicates()
gc.collect()

133

In [ ]:
user1 = feature3[['user_id','sku_id','price','o_id','o_date','o_sku_num','o_month','o_day','comment_create_tm','score_level','comment_year','comment_month','comment_day','age','user_lv_cd','a_date','a_num','a_type','a_year','a_month','a_day']]
t = user1[['user_id','age','user_lv_cd','cate']]
t.drop_duplicates(inplace=True)
###m1
sample_user1 = feat_nunique(t,user1[(user1.cate==101)|(user1.cate==30)],['sku_id'],'o_id','f3_buytimes_30101_')
sample_user1 = feat_nunique(sample_user1,user1[(user1.cate==30)],['sku_id'],'o_id','f3_buytimes_30_')
sample_user1 = feat_nunique(sample_user1,user1[(user1.cate==101)],['sku_id'],'o_id','f3_buytimes_101_')
sample_user1 = feat_nunique(sample_user1,user1[(user1.cate!=101)&(user1.cate!=30)],['sku_id'],'o_id','f3_buytimes_no_30101_')

###m2-m4
sample_user1['f3_buytimes_30101_in_7days_'] = sample_user1['f3_buytimes_30101_o_id_sku_id_nunique']/12
sample_user1['f3_buytimes_30_in_7days_'] = sample_user1['f3_buytimes_30_o_id_sku_id_nunique']/12
sample_user1['f3_buytimes_101_in_7days_'] = sample_user1['f3_buytimes_101_o_id_sku_id_nunique']/12
sample_user1['f3_buytimes_no_30101_in_7days_'] = sample_user1['f3_buytimes_no_30101_o_id_sku_id_nunique']/12

sample_user1['f3_buytimes_30101_in_15days_'] = sample_user1['f3_buytimes_30101_o_id_sku_id_nunique']/6
sample_user1['f3_buytimes_30_in_15days_'] = sample_user1['f3_buytimes_30_o_id_sku_id_nunique']/6
sample_user1['f3_buytimes_101_in_15days_'] = sample_user1['f3_buytimes_101_o_id_sku_id_nunique']/6
sample_user1['f3_buytimes_no_30101_in_15days_'] = sample_user1['f3_buytimes_no_30101_o_id_sku_id_nunique']/6

sample_user1['f3_buytimes_30101_in_31days_'] = sample_user1['f3_buytimes_30101_o_id_sku_id_nunique']/3
sample_user1['f3_buytimes_30_in_31days_'] = sample_user1['f3_buytimes_30_o_id_sku_id_nunique']/3
sample_user1['f3_buytimes_101_in_31days_'] = sample_user1['f3_buytimes_101_o_id_sku_id_nunique']/3
sample_user1['f3_buytimes_no_30101_in_31days_'] = sample_user1['f3_buytimes_no_30101_o_id_sku_id_nunique']/3

###m5
sample_user1 = feat_sum(sample_user1,user1[(user1.cate==101)|(user1.cate==30)],['sku_id'],'o_sku_num','f3_buynums_30101_')
sample_user1 = feat_sum(sample_user1,user1[(user1.cate==30)],['sku_id'],'o_sku_num','f3_buynums_30_')
sample_user1 = feat_sum(sample_user1,user1[(user1.cate==101)],['sku_id'],'o_sku_num','f3_buynums_101_')
sample_user1 = feat_sum(sample_user1,user1[(user1.cate!=101)&(user1.cate!=30)],['sku_id'],'o_sku_num','f3_buynums_no_30101_')

###m6-m8
sample_user1['f3_buynums_30101_in_7days_'] = sample_user1['f3_buynums_30101_o_sku_num_sku_id_sum']/12
sample_user1['f3_buynums_30_in_7days_'] = sample_user1['f3_buynums_30_o_sku_num_sku_id_sum']/12
sample_user1['f3_buynums_101_in_7days_'] = sample_user1['f3_buynums_101_o_sku_num_sku_id_sum']/12
sample_user1['f3_buynums_no_30101_in_7days_'] = sample_user1['f3_buynums_no_30101_o_sku_num_sku_id_sum']/12

sample_user1['f3_buynums_30101_in_15days_'] = sample_user1['f3_buynums_30101_o_sku_num_sku_id_sum']/6
sample_user1['f3_buynums_30_in_15days_'] = sample_user1['f3_buynums_30_o_sku_num_sku_id_sum']/6
sample_user1['f3_buynums_101_in_15days_'] = sample_user1['f3_buynums_101_o_sku_num_sku_id_sum']/6
sample_user1['f3_buynums_no_30101_in_15days_'] = sample_user1['f3_buynums_no_30101_o_sku_num_sku_id_sum']/6

sample_user1['f3_buynums_30101_in_31days_'] = sample_user1['f3_buynums_30101_o_sku_num_sku_id_sum']/3
sample_user1['f3_buynums_30_in_31days_'] = sample_user1['f3_buynums_30_o_sku_num_sku_id_sum']/3
sample_user1['f3_buynums_101_in_31days_'] = sample_user1['f3_buynums_101_o_sku_num_sku_id_sum']/3
sample_user1['f3_buynums_no_30101_in_31days_'] = sample_user1['f3_buynums_no_30101_o_sku_num_sku_id_sum']/3

###m9
sample_user1 = feat_sum(sample_user1,user1[((user1.cate==101)|(user1.cate==30))&(user1.a_type==1)],['sku_id'],'a_num','f3_scannums_30101_')
sample_user1 = feat_sum(sample_user1,user1[((user1.cate==30))&(user1.a_type==1)],['sku_id'],'a_num','f3_scannums_30_')
sample_user1 = feat_sum(sample_user1,user1[((user1.cate==101))&(user1.a_type==1)],['sku_id'],'a_num','f3_scannums_101_')
sample_user1 = feat_sum(sample_user1,user1[((user1.cate!=101)&(user1.cate!=30))&(user1.a_type==1)],['sku_id'],'a_num','f3_scannums_no_30101_')
###m10-m12
sample_user1['f3_scannums_30101_in_7days_'] = sample_user1['f3_scannums_30101_a_num_sku_id_sum']/12
sample_user1['f3_scannums_30_in_7days_'] = sample_user1['f3_scannums_30_a_num_sku_id_sum']/12
sample_user1['f3_scannums_101_in_7days_'] = sample_user1['f3_scannums_101_a_num_sku_id_sum']/12
sample_user1['f3_scannums_no_30101_in_7days_'] = sample_user1['f3_scannums_no_30101_a_num_sku_id_sum']/12

sample_user1['f3_scannums_30101_in_15days_'] = sample_user1['f3_scannums_30101_a_num_sku_id_sum']/6
sample_user1['f3_scannums_30_in_15days_'] = sample_user1['f3_scannums_30_a_num_sku_id_sum']/6
sample_user1['f3_scannums_101_in_15days_'] = sample_user1['f3_scannums_101_a_num_sku_id_sum']/6
sample_user1['f3_scannums_no_30101_in_15days_'] = sample_user1['f3_scannums_no_30101_a_num_sku_id_sum']/6

sample_user1['f3_scannums_30101_in_31days_'] = sample_user1['f3_scannums_30101_a_num_sku_id_sum']/3
sample_user1['f3_scannums_30_in_31days_'] = sample_user1['f3_scannums_30_a_num_sku_id_sum']/3
sample_user1['f3_scannums_101_in_31days_'] = sample_user1['f3_scannums_101_a_num_sku_id_sum']/3
sample_user1['f3_scannums_no_30101_in_31days_'] = sample_user1['f3_scannums_no_30101_a_num_sku_id_sum']/3


###m13
sample_user1 = feat_sum(sample_user1,user1[((user1.cate==101)|(user1.cate==30))&(user1.a_type==2)],['sku_id'],'a_num','f3_attnums_30101_')
sample_user1 = feat_sum(sample_user1,user1[((user1.cate==30))&(user1.a_type==2)],['sku_id'],'a_num','f3_attnums_30_')
sample_user1 = feat_sum(sample_user1,user1[((user1.cate==101))&(user1.a_type==2)],['sku_id'],'a_num','f3_attnums_101_')
sample_user1 = feat_sum(sample_user1,user1[((user1.cate!=101)&(user1.cate!=30))&(user1.a_type==2)],['sku_id'],'a_num','f3_attnums_no_30101_')
###m14-m16
sample_user1['f3_attnums_30101_in_7days_'] = sample_user1['f3_attnums_30101_a_num_sku_id_sum']/12
sample_user1['f3_attnums_30_in_7days_'] = sample_user1['f3_attnums_30_a_num_sku_id_sum']/12
sample_user1['f3_attnums_101_in_7days_'] = sample_user1['f3_attnums_101_a_num_sku_id_sum']/12
sample_user1['f3_attnums_no_30101_in_7days_'] = sample_user1['f3_attnums_no_30101_a_num_sku_id_sum']/12

sample_user1['f3_attnums_30101_in_15days_'] = sample_user1['f3_attnums_30101_a_num_sku_id_sum']/6
sample_user1['f3_attnums_30_in_15days_'] = sample_user1['f3_attnums_30_a_num_sku_id_sum']/6
sample_user1['f3_attnums_101_in_15days_'] = sample_user1['f3_attnums_101_a_num_sku_id_sum']/6
sample_user1['f3_attnums_no_30101_in_15days_'] = sample_user1['f3_attnums_no_30101_a_num_sku_id_sum']/6

sample_user1['f3_attnums_30101_in_31days_'] = sample_user1['f3_attnums_30101_a_num_sku_id_sum']/3
sample_user1['f3_attnums_30_in_31days_'] = sample_user1['f3_attnums_30_a_num_sku_id_sum']/3
sample_user1['f3_attnums_101_in_31days_'] = sample_user1['f3_attnums_101_a_num_sku_id_sum']/3
sample_user1['f3_attnums_no_30101_in_31days_'] = sample_user1['f3_attnums_no_30101_a_num_sku_id_sum']/3



###m17
sample_user1 = feat_nunique(sample_user1,user1[(user1.cate==101)|(user1.cate==30)],['sku_id'],'comment_create_tm','f3_commentnums_30101_')
sample_user1 = feat_nunique(sample_user1,user1[(user1.cate==30)],['sku_id'],'comment_create_tm','f3_commentnums_30_')
sample_user1 = feat_nunique(sample_user1,user1[(user1.cate==101)],['sku_id'],'comment_create_tm','f3_commentnums_101_')
sample_user1 = feat_nunique(sample_user1,user1[(user1.cate!=101)&(user1.cate!=30)],['sku_id'],'comment_create_tm','f3_commentnums_no_30101_')

###m18-20
sample_user1['f3_commentnums_30101_in_7days_'] = sample_user1['f3_commentnums_30101_comment_create_tm_sku_id_nunique']/12
sample_user1['f3_commentnums_30_in_7days_'] = sample_user1['f3_commentnums_30_comment_create_tm_sku_id_nunique']/12
sample_user1['f3_commentnums_101_in_7days_'] = sample_user1['f3_commentnums_101_comment_create_tm_sku_id_nunique']/12
sample_user1['f3_commentnums_no_30101_in_7days_'] = sample_user1['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']/12

sample_user1['f3_commentnums_30101_in_15days_'] = sample_user1['f3_commentnums_30101_comment_create_tm_sku_id_nunique']/6
sample_user1['f3_commentnums_30_in_15days_'] = sample_user1['f3_commentnums_30_comment_create_tm_sku_id_nunique']/6
sample_user1['f3_commentnums_101_in_15days_'] = sample_user1['f3_commentnums_101_comment_create_tm_sku_id_nunique']/6
sample_user1['f3_commentnums_no_30101_in_15days_'] = sample_user1['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']/6

sample_user1['f3_commentnums_30101_in_31days_'] = sample_user1['f3_commentnums_30101_comment_create_tm_sku_id_nunique']/3
sample_user1['f3_commentnums_30_in_31days_'] = sample_user1['f3_commentnums_30_comment_create_tm_sku_id_nunique']/3
sample_user1['f3_commentnums_101_in_31days_'] = sample_user1['f3_commentnums_101_comment_create_tm_sku_id_nunique']/3
sample_user1['f3_commentnums_no_30101_in_31days_'] = sample_user1['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']/3

###m21
t1 = sample_user1[['sku_id','f3_buytimes_30101_o_id_sku_id_nunique']]
t2 = user1[(user1.cate==101)|(user1.cate==30)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.cate==101)|(user1.cate==30)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_30101_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_30101_buy_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user1[['sku_id','f3_buytimes_30_o_id_sku_id_nunique']]
t2 = user1[(user1.cate==30)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.cate==30)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_30_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_30_buy_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user1[['sku_id','f3_buytimes_101_o_id_sku_id_nunique']]
t2 = user1[(user1.cate==101)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.cate==101)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_101_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_101_buy_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user1[['sku_id','f3_buytimes_no_30101_o_id_sku_id_nunique']]
t2 = user1[(user1.cate!=101)&(user1.cate!=30)].groupby(['sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.cate!=101)&(user1.cate!=30)].groupby(['sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','o_date':'o_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_no_30101_o_id_sku_id_nunique']
t = t[['sku_id','f3_average_no_30101_buy_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')

###m22
t1 = sample_user1[['sku_id','f3_scannums_30101_a_num_sku_id_sum']]
t2 = user1[(user1.a_type==1)&((user1.cate==101)|(user1.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.a_type==1)&((user1.cate==101)|(user1.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30101_scan_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user1[['sku_id','f3_scannums_30_a_num_sku_id_sum']]
t2 = user1[(user1.a_type==1)&((user1.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.a_type==1)&((user1.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_30_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30_scan_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user1[['sku_id','f3_scannums_101_a_num_sku_id_sum']]
t2 = user1[(user1.a_type==1)&((user1.cate==101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.a_type==1)&((user1.cate==101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_101_scan_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user1[['sku_id','f3_scannums_no_30101_a_num_sku_id_sum']]
t2 = user1[(user1.a_type==1)&((user1.cate!=30)&(user1.cate!=101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.a_type==1)&((user1.cate!=30)&(user1.cate!=101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_no_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_no_30101_scan_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')


###m23
t1 = sample_user1[['sku_id','f3_attnums_30101_a_num_sku_id_sum']]
t2 = user1[(user1.a_type==2)&((user1.cate==101)|(user1.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.a_type==2)&((user1.cate==101)|(user1.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30101_att_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user1[['sku_id','f3_attnums_30_a_num_sku_id_sum']]
t2 = user1[(user1.a_type==2)&((user1.cate==30))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.a_type==2)&((user1.cate==30))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_30_a_num_sku_id_sum']
t = t[['sku_id','f3_average_30_att_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user1[['sku_id','f3_attnums_101_a_num_sku_id_sum']]
t2 = user1[(user1.a_type==2)&((user1.cate==101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.a_type==2)&((user1.cate==101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_101_att_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')

###
t1 = sample_user1[['sku_id','f3_attnums_no_30101_a_num_sku_id_sum']]
t2 = user1[(user1.a_type==2)&((user1.cate!=30)&(user1.cate!=101))].groupby(['sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.a_type==2)&((user1.cate!=30)&(user1.cate!=101))].groupby(['sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','a_date':'a_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_no_30101_a_num_sku_id_sum']
t = t[['sku_id','f3_average_no_30101_att_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')


###m24
t1 = sample_user1[['sku_id','f3_commentnums_30101_comment_create_tm_sku_id_nunique']]
t2 = user1[(user1.cate==101)|(user1.cate==30)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.cate==101)|(user1.cate==30)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_30101_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_30101_comment_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')

t1 = sample_user1[['sku_id','f3_commentnums_30_comment_create_tm_sku_id_nunique']]
t2 = user1[(user1.cate==30)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.cate==30)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_30_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_30_comment_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')


t1 = sample_user1[['sku_id','f3_commentnums_101_comment_create_tm_sku_id_nunique']]
t2 = user1[(user1.cate==101)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.cate==101)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_101_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_101_comment_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')


t1 = sample_user1[['sku_id','f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']]
t2 = user1[(user1.cate!=101)&(user1.cate!=30)].groupby(['sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['sku_id'],how='left')
t3 = user1[(user1.cate!=101)&(user1.cate!=30)].groupby(['sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_no_30101_comment_create_tm_sku_id_nunique']
t = t[['sku_id','f3_average_no_30101_comment_day_gap']]
sample_user1 = sample_user1.merge(t,on=['sku_id'],how='left')

###m25-m32还没写，正在考虑
user1_a_o_date = user1[((pd.to_datetime(user1.a_date)<=pd.to_datetime(user1.o_date.fillna('2018-01-01'))))]
user1_a_o_date = user1_a_o_date.sort_values(by=['sku_id','a_date','o_date']).drop_duplicates(['sku_id','a_date'],keep="first")
user1_a_o_date = user1_a_o_date[['sku_id','a_date','o_date','cate']]
user1_a_o_date['o_a_gap']=(pd.to_datetime(user1_a_o_date.o_date)-pd.to_datetime(user1_a_o_date.a_date)).dt.days 
FLAG='user1_a_o_'
sample_user1 = feat_count(sample_user1,user1_a_o_date[user1_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG+'order_')
sample_user1 = feat_mean(sample_user1,user1_a_o_date[(user1_a_o_date.o_a_gap>=-1)&((user1_a_o_date.cate==101)|(user1_a_o_date.cate==30))],['sku_id'],'o_a_gap',FLAG+'30101_')
sample_user1 = feat_mean(sample_user1,user1_a_o_date[(user1_a_o_date.o_a_gap>=-1)&((user1_a_o_date.cate==30))],['sku_id'],'o_a_gap',FLAG+'30_')
sample_user1 = feat_mean(sample_user1,user1_a_o_date[(user1_a_o_date.o_a_gap>=-1)&((user1_a_o_date.cate==101))],['sku_id'],'o_a_gap',FLAG+'101_')
sample_user1 = feat_mean(sample_user1,user1_a_o_date[(user1_a_o_date.o_a_gap>=-1)&((user1_a_o_date.cate!=101)&(user1_a_o_date.cate!=30))],['sku_id'],'o_a_gap',FLAG+'no_30101_')


###u33
t1 = user1[['user_id','o_sku_num','price','cate']] 
t1['sku_total_price'] = t1['o_sku_num'] * t1['price']
sample_user1 = feat_sum(sample_user1,t1[(t1.cate==30)|(t1.cate==101)],['user_id'],'sku_total_price','f3_user_total_consumption_30101_')
sample_user1 = feat_sum(sample_user1,t1[(t1.cate==30)],['user_id'],'sku_total_price','f3_user_total_consumption_30_')
sample_user1 = feat_sum(sample_user1,t1[(t1.cate==101)],['user_id'],'sku_total_price','f3_user_total_consumption_101_')
sample_user1 = feat_sum(sample_user1,t1[(t1.cate!=30)&(t1.cate!=101)],['user_id'],'sku_total_price','f3_user_total_consumption_no_30101_')

###u34-u36
sample_user1['f3_total_consumption_30101_in_7days_'] = sample_user1['f3_user_total_consumption_30101_sku_total_price_user_id_sum']/12
sample_user1['f3_total_consumption_30_in_7days_'] = sample_user1['f3_user_total_consumption_30_sku_total_price_user_id_sum']/12
sample_user1['f3_total_consumption_101_in_7days_'] = sample_user1['f3_user_total_consumption_101_sku_total_price_user_id_sum']/12
sample_user1['f3_total_consumption_no_30101_in_7days_'] = sample_user1['f3_user_total_consumption_no_30101_sku_total_price_user_id_sum']/12

sample_user1['f3_total_consumption_30101_in_15days_'] = sample_user1['f3_user_total_consumption_30101_sku_total_price_user_id_sum']/6
sample_user1['f3_total_consumption_30_in_15days_'] = sample_user1['f3_user_total_consumption_30_sku_total_price_user_id_sum']/6
sample_user1['f3_total_consumption_101_in_15days_'] = sample_user1['f3_user_total_consumption_101_sku_total_price_user_id_sum']/6
sample_user1['f3_total_consumption_no_30101_in_15days_'] = sample_user1['f3_user_total_consumption_no_30101_sku_total_price_user_id_sum']/6

sample_user1['f3_total_consumption_30101_in_31days_'] = sample_user1['f3_user_total_consumption_30101_sku_total_price_user_id_sum']/3
sample_user1['f3_total_consumption_30_in_31days_'] = sample_user1['f3_user_total_consumption_30_sku_total_price_user_id_sum']/3
sample_user1['f3_total_consumption_101_in_31days_'] = sample_user1['f3_user_total_consumption_101_sku_total_price_user_id_sum']/3
sample_user1['f3_total_consumption_no_30101_in_31days_'] = sample_user1['f3_user_total_consumption_no_30101_sku_total_price_user_id_sum']/3


###u37-u40
sample_user1 = feat_min(sample_user1,user1[()],['user_id'],'o_day','f3_buytime_early_')
sample_user1 = feat_min(sample_user1,user1[user1.a_type==1],['user_id'],'a_day','f3_scantime_early_')
sample_user1 = feat_min(sample_user1,user1[user1.a_type==2],['user_id'],'a_day','f3_atttime_early_')
sample_user1 = feat_min(sample_user1,user1,['user_id'],'comment_day','f3_commenttime_early_')

sample_user1 = feat_min(sample_user1,user1[(user1.cate==101)|(user1.cate==30)],['user_id'],'o_day','f3_buytime_early_30101_')
sample_user1 = feat_min(sample_user1,user1[(user1.cate==30)],['user_id'],'o_day','f3_buytime_early_30_')
sample_user1 = feat_min(sample_user1,user1[(user1.cate==101)],['user_id'],'o_day','f3_buytime_early_101_')
sample_user1 = feat_min(sample_user1,user1[(user1.cate!=101)&(user1.cate!=30)],['user_id'],'o_day','f3_buytime_early_no_30101_')

sample_user1 = feat_min(sample_user1,user1[((user1.cate==101)|(user1.cate==30))&(user1.a_type==1)],['user_id'],'a_day','f3_scantime_early_30101_')
sample_user1 = feat_min(sample_user1,user1[(user1.cate==30)&(user1.a_type==1)],['user_id'],'a_day','f3_scantime_early_30_')
sample_user1 = feat_min(sample_user1,user1[(user1.cate==101)&(user1.a_type==1)],['user_id'],'a_day','f3_scantime_early_101_')
sample_user1 = feat_min(sample_user1,user1[((user1.cate!=101)&(user1.cate!=30))&(user1.a_type==1)],['user_id'],'a_day','f3_scantime_early_30101_')

sample_user1 = feat_min(sample_user1,user1[((user1.cate==101)|(user1.cate==30))&(user1.a_type==2)],['user_id'],'a_day','f3_atttime_early_30101_')
sample_user1 = feat_min(sample_user1,user1[(user1.cate==30)&(user1.a_type==2)],['user_id'],'a_day','f3_atttime_early_30_')
sample_user1 = feat_min(sample_user1,user1[(user1.cate==101)&(user1.a_type==2)],['user_id'],'a_day','f3_atttime_early_101_')
sample_user1 = feat_min(sample_user1,user1[((user1.cate!=101)&(user1.cate!=30))&(user1.a_type==2)],['user_id'],'a_day','f3_atttime_early_no_30101_')

sample_user1 = feat_min(sample_user1,user1[(user1.cate==101)|(user1.cate==30)],['user_id'],'comment_day','f3_commenttime_early_30101_')
sample_user1 = feat_min(sample_user1,user1[(user1.cate==30)],['user_id'],'comment_day','f3_commenttime_early_30_')
sample_user1 = feat_min(sample_user1,user1[(user1.cate==101)],['user_id'],'comment_day','f3_commenttime_early_101_')
sample_user1 = feat_min(sample_user1,user1[(user1.cate!=101)&(user1.cate!=30)],['user_id'],'comment_day','f3_commenttime_early_no_30101_')


###u41-u44
sample_user1 = feat_max(sample_user1,user1[(user1.cate==101)|(user1.cate==30)],['user_id'],'o_day','f3_buytime_late_30101_')
sample_user1 = feat_max(sample_user1,user1[(user1.cate==30)],['user_id'],'o_day','f3_buytime_late_30_')
sample_user1 = feat_max(sample_user1,user1[(user1.cate==101)],['user_id'],'o_day','f3_buytime_late_101_')
sample_user1 = feat_max(sample_user1,user1[(user1.cate!=101)&(user1.cate!=30)],['user_id'],'o_day','f3_buytime_late_no_30101_')

sample_user1 = feat_max(sample_user1,user1[((user1.cate==101)|(user1.cate==30))&(user1.a_type==1)],['user_id'],'a_day','f3_scantime_late_30101_')
sample_user1 = feat_max(sample_user1,user1[(user1.cate==30)&(user1.a_type==1)],['user_id'],'a_day','f3_scantime_late_30_')
sample_user1 = feat_max(sample_user1,user1[(user1.cate==101)&(user1.a_type==1)],['user_id'],'a_day','f3_scantime_late_101_')
sample_user1 = feat_max(sample_user1,user1[((user1.cate!=101)&(user1.cate!=30))&(user1.a_type==1)],['user_id'],'a_day','f3_scantime_late_30101_')

sample_user1 = feat_max(sample_user1,user1[((user1.cate==101)|(user1.cate==30))&(user1.a_type==2)],['user_id'],'a_day','f3_atttime_late_30101_')
sample_user1 = feat_max(sample_user1,user1[(user1.cate==30)&(user1.a_type==2)],['user_id'],'a_day','f3_atttime_late_30_')
sample_user1 = feat_max(sample_user1,user1[(user1.cate==101)&(user1.a_type==2)],['user_id'],'a_day','f3_atttime_late_101_')
sample_user1 = feat_max(sample_user1,user1[((user1.cate!=101)&(user1.cate!=30))&(user1.a_type==2)],['user_id'],'a_day','f3_atttime_late_no_30101_')

sample_user1 = feat_max(sample_user1,user1[(user1.cate==101)|(user1.cate==30)],['user_id'],'comment_day','f3_commenttime_late_30101_')
sample_user1 = feat_max(sample_user1,user1[(user1.cate==30)],['user_id'],'comment_day','f3_commenttime_late_30_')
sample_user1 = feat_max(sample_user1,user1[(user1.cate==101)],['user_id'],'comment_day','f3_commenttime_late_101_')
sample_user1 = feat_max(sample_user1,user1[(user1.cate!=101)&(user1.cate!=30)],['user_id'],'comment_day','f3_commenttime_late_no_30101_')


sample_user1 = sample_user1.drop_duplicates()
gc.collect()


In [ ]:
###用户-商品组合特征
#####dataset3_user_sku
#####
###for dataset3
user_sku3 = feature3[(feature3.cate==30)|(feature3.cate==101)]
t = user_sku3[['user_id','sku_id']]
t.drop_duplicates(inplace=True)
###um1
sample_user_sku3 = feat_nunique(t,user_sku3,['user_id','sku_id'],'o_id','f3_buytimes_')
###um2-um4
sample_user_sku3['f3_buytimes_in_7days_'] = sample_user_sku3['f3_buytimes_o_id_user_id_sku_id_nunique']/12
sample_user_sku3['f3_buytimes_in_15days_'] = sample_user_sku3['f3_buytimes_o_id_user_id_sku_id_nunique']/6
sample_user_sku3['f3_buytimes_in_31days_'] = sample_user_sku3['f3_buytimes_o_id_user_id_sku_id_nunique']/3

###um5
sample_user_sku3 = feat_sum(sample_user_sku3,user_sku3,['user_id','sku_id'],'o_sku_num','f3_buynums_')
###um6-um8
sample_user_sku3['f3_buynums_in_7days_'] = sample_user_sku3['f3_buynums_o_sku_num_user_id_sku_id_sum']/12
sample_user_sku3['f3_buynums_in_15days_'] = sample_user_sku3['f3_buynums_o_sku_num_user_id_sku_id_sum']/6
sample_user_sku3['f3_buynums_in_31days_'] = sample_user_sku3['f3_buynums_o_sku_num_user_id_sku_id_sum']/3

###um9
sample_user_sku3 = feat_sum(sample_user_sku3,user_sku3[user_sku3.a_type==1],['user_id','sku_id'],'a_num','f3_scannums_')
###um10-um12
sample_user_sku3['f3_scannums_in_7days_'] = sample_user_sku3['f3_scannums_a_num_user_id_sku_id_sum']/12
sample_user_sku3['f3_scannums_in_15days_'] = sample_user_sku3['f3_scannums_a_num_user_id_sku_id_sum']/6
sample_user_sku3['f3_scannums_in_31days_'] = sample_user_sku3['f3_scannums_a_num_user_id_sku_id_sum']/3

###um13
sample_user_sku3 = feat_sum(sample_user_sku3,user_sku3[user_sku3.a_type==2],['user_id','sku_id'],'a_num','f3_attnums_')
###um14-um16
sample_user_sku3['f3_attnums_in_7days_'] = sample_user_sku3['f3_attnums_a_num_user_id_sku_id_sum']/12
sample_user_sku3['f3_attnums_in_15days_'] = sample_user_sku3['f3_attnums_a_num_user_id_sku_id_sum']/6
sample_user_sku3['f3_attnums_in_31days_'] = sample_user_sku3['f3_attnums_a_num_user_id_sku_id_sum']/3

###um17
sample_user_sku3 = feat_nunique(sample_user_sku3,user_sku3,['user_id','sku_id'],'comment_create_tm','f3_commentnums_')
###um18-20
sample_user_sku3['f3_commentnums_in_7days_'] = sample_user_sku3['f3_commentnums_comment_create_tm_user_id_sku_id_nunique']/12
sample_user_sku3['f3_commentnums_in_15days_'] = sample_user_sku3['f3_commentnums_comment_create_tm_user_id_sku_id_nunique']/6
sample_user_sku3['f3_commentnums_in_31days_'] = sample_user_sku3['f3_commentnums_comment_create_tm_user_id_sku_id_nunique']/3

###um21
t1 = sample_user_sku3[['user_id','f3_buytimes_o_id_user_id_sku_id_nunique','sku_id']]
t2 = user_sku3.groupby(['user_id','sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['user_id','sku_id'],how='left')
t3 = user_sku3.groupby(['user_id','sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','o_date':'o_date_min'})
t = t.merge(t3,on=['user_id','sku_id'],how='left')
t['f3_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f3_buytimes_o_id_user_id_sku_id_nunique']
t = t[['user_id','sku_id','f3_average_buy_day_gap']]
sample_user_sku3 = sample_user_sku3.merge(t,on=['user_id','sku_id'],how='left')

###um22
t1 = sample_user_sku3[['user_id','f3_scannums_a_num_user_id_sku_id_sum','sku_id']]
t2 = user_sku3[user_sku3.a_type==1].groupby(['user_id','sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['user_id','sku_id'],how='left')
t3 = user_sku3[user_sku3.a_type==1].groupby(['user_id','sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','a_date':'a_date_min'})
t = t.merge(t3,on=['user_id','sku_id'],how='left')
t['f3_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_scannums_a_num_user_id_sku_id_sum']
t = t[['user_id','sku_id','f3_average_scan_day_gap']]
sample_user_sku3 = sample_user_sku3.merge(t,on=['user_id','sku_id'],how='left')


###um23
t1 = sample_user_sku3[['user_id','sku_id','f3_attnums_a_num_user_id_sku_id_sum']]
t2 = user_sku3[user_sku3.a_type==2].groupby(['user_id','sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['user_id','sku_id'],how='left')
t3 = user_sku3[user_sku3.a_type==2].groupby(['user_id','sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','a_date':'a_date_min'})
t = t.merge(t3,on=['user_id','sku_id'],how='left')
t['f3_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f3_attnums_a_num_user_id_sku_id_sum']
t = t[['user_id','sku_id','f3_average_att_day_gap']]
sample_user_sku3 = sample_user_sku3.merge(t,on=['user_id','sku_id'],how='left')


###um24
t1 = sample_user_sku3[['user_id','sku_id','f3_commentnums_comment_create_tm_user_id_sku_id_nunique']]
t2 = user_sku3.groupby(['user_id','sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['user_id','sku_id'],how='left')
t3 = user_sku3.groupby(['user_id','sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['user_id','sku_id'],how='left')
t['f3_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f3_commentnums_comment_create_tm_user_id_sku_id_nunique']
t = t[['user_id','sku_id','f3_average_comment_day_gap']]
sample_user_sku3 = sample_user_sku3.merge(t,on=['user_id','sku_id'],how='left')


###um25-um32还没写，正在考虑
user_sku3_a_o_date = user_sku3[((pd.to_datetime(user_sku3.a_date)<=pd.to_datetime(user_sku3.o_date.fillna('2018-01-01'))))]
user_sku3_a_o_date = user_sku3_a_o_date.sort_values(by=['sku_id','a_date','o_date']).drop_duplicates(['sku_id','a_date'],keep="first")
user_sku3_a_o_date = user_sku3_a_o_date[['sku_id','a_date','o_date','cate']]
user_sku3_a_o_date['o_a_gap']=(pd.to_datetime(user_sku3_a_o_date.o_date)-pd.to_datetime(user_sku3_a_o_date.a_date)).dt.days 
FLAG='user_sku3_a_o_'
sample_user_sku3 = feat_count(sample_user_sku3,user_sku3_a_o_date[user_sku3_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG+'order_')
sample_user_sku3 = feat_mean(sample_user_sku3,user_sku3_a_o_date[user_sku3_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG)

###um33
t1 = user_sku3[['user_id','sku_id','o_sku_num','price']] 
t1['user_sku_total_price'] = t1['o_sku_num'] * t1['price']
sample_user_sku3 = feat_sum(sample_user_sku3,t1,['user_id','sku_id'],'user_sku_total_price','f3_user_total_consumption_')

###um34-um36
sample_user_sku3['f3_total_consumption_in_7days_'] = sample_user_sku3['f3_user_total_consumption_user_sku_total_price_user_id_sku_id_sum']/12
sample_user_sku3['f3_total_consumption_in_15days_'] = sample_user_sku3['f3_user_total_consumption_user_sku_total_price_user_id_sku_id_sum']/6
sample_user_sku3['f3_total_consumption_in_31days_'] = sample_user_sku3['f3_user_total_consumption_user_sku_total_price_user_id_sku_id_sum']/3

###um37-um40
sample_user_sku3 = feat_min(sample_user_sku3,user_sku3,['user_id','sku_id'],'o_day','f3_buytime_early_')
sample_user_sku3 = feat_min(sample_user_sku3,user_sku3[user_sku3.a_type==1],['user_id','sku_id'],'a_day','f3_scantime_early_')
sample_user_sku3 = feat_min(sample_user_sku3,user_sku3[user_sku3.a_type==2],['user_id','sku_id'],'a_day','f3_atttime_early_')
sample_user_sku3 = feat_min(sample_user_sku3,user_sku3,['user_id','sku_id'],'comment_day','f3_commenttime_early_')

###um41-um44
sample_user_sku3 = feat_max(sample_user_sku3,user_sku3,['user_id','sku_id'],'o_day','f3_buytime_late_')
sample_user_sku3 = feat_max(sample_user_sku3,user_sku3[user_sku3.a_type==1],['user_id','sku_id'],'a_day','f3_scantime_late_')
sample_user_sku3 = feat_max(sample_user_sku3,user_sku3[user_sku3.a_type==2],['user_id','sku_id'],'a_day','f3_atttime_late_')
sample_user_sku3 = feat_max(sample_user_sku3,user_sku3,['user_id','sku_id'],'comment_day','f3_commenttime_late_')


sample_user_sku3 = sample_user_sku3.drop_duplicates()
gc.collect()

In [ ]:
###用户-商品组合特征
#####dataset2_user_sku
#####
###for dataset2
user_sku2 = feature2[(feature2.cate==30)|(feature2.cate==101)]
t = user_sku2[['user_id','sku_id']]
t.drop_duplicates(inplace=True)
###um1
sample_user_sku2 = feat_nunique(t,user_sku2,['user_id','sku_id'],'o_id','f2_buytimes_')
###um2-um4
sample_user_sku2['f2_buytimes_in_7days_'] = sample_user_sku2['f2_buytimes_o_id_user_id_sku_id_nunique']/12
sample_user_sku2['f2_buytimes_in_15days_'] = sample_user_sku2['f2_buytimes_o_id_user_id_sku_id_nunique']/6
sample_user_sku2['f2_buytimes_in_31days_'] = sample_user_sku2['f2_buytimes_o_id_user_id_sku_id_nunique']/3

###um5
sample_user_sku2 = feat_sum(sample_user_sku2,user_sku2,['user_id','sku_id'],'o_sku_num','f2_buynums_')
###um6-um8
sample_user_sku2['f2_buynums_in_7days_'] = sample_user_sku2['f2_buynums_o_sku_num_user_id_sku_id_sum']/12
sample_user_sku2['f2_buynums_in_15days_'] = sample_user_sku2['f2_buynums_o_sku_num_user_id_sku_id_sum']/6
sample_user_sku2['f2_buynums_in_31days_'] = sample_user_sku2['f2_buynums_o_sku_num_user_id_sku_id_sum']/3

###um9
sample_user_sku2 = feat_sum(sample_user_sku2,user_sku2[user_sku2.a_type==1],['user_id','sku_id'],'a_num','f2_scannums_')
###um10-um12
sample_user_sku2['f2_scannums_in_7days_'] = sample_user_sku2['f2_scannums_a_num_user_id_sku_id_sum']/12
sample_user_sku2['f2_scannums_in_15days_'] = sample_user_sku2['f2_scannums_a_num_user_id_sku_id_sum']/6
sample_user_sku2['f2_scannums_in_31days_'] = sample_user_sku2['f2_scannums_a_num_user_id_sku_id_sum']/3

###um13
sample_user_sku2 = feat_sum(sample_user_sku2,user_sku2[user_sku2.a_type==2],['user_id','sku_id'],'a_num','f2_attnums_')
###um14-um16
sample_user_sku2['f2_attnums_in_7days_'] = sample_user_sku2['f2_attnums_a_num_user_id_sku_id_sum']/12
sample_user_sku2['f2_attnums_in_15days_'] = sample_user_sku2['f2_attnums_a_num_user_id_sku_id_sum']/6
sample_user_sku2['f2_attnums_in_31days_'] = sample_user_sku2['f2_attnums_a_num_user_id_sku_id_sum']/3

###um17
sample_user_sku2 = feat_nunique(sample_user_sku2,user_sku2,['user_id','sku_id'],'comment_create_tm','f2_commentnums_')
###um18-20
sample_user_sku2['f2_commentnums_in_7days_'] = sample_user_sku2['f2_commentnums_comment_create_tm_user_id_sku_id_nunique']/12
sample_user_sku2['f2_commentnums_in_15days_'] = sample_user_sku2['f2_commentnums_comment_create_tm_user_id_sku_id_nunique']/6
sample_user_sku2['f2_commentnums_in_31days_'] = sample_user_sku2['f2_commentnums_comment_create_tm_user_id_sku_id_nunique']/3

###um21
t1 = sample_user_sku2[['user_id','f2_buytimes_o_id_user_id_sku_id_nunique','sku_id']]
t2 = user_sku2.groupby(['user_id','sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['user_id','sku_id'],how='left')
t3 = user_sku2.groupby(['user_id','sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','o_date':'o_date_min'})
t = t.merge(t3,on=['user_id','sku_id'],how='left')
t['f2_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f2_buytimes_o_id_user_id_sku_id_nunique']
t = t[['user_id','sku_id','f2_average_buy_day_gap']]
sample_user_sku2 = sample_user_sku2.merge(t,on=['user_id','sku_id'],how='left')

###um22
t1 = sample_user_sku2[['user_id','f2_scannums_a_num_user_id_sku_id_sum','sku_id']]
t2 = user_sku2[user_sku2.a_type==1].groupby(['user_id','sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['user_id','sku_id'],how='left')
t3 = user_sku2[user_sku2.a_type==1].groupby(['user_id','sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','a_date':'a_date_min'})
t = t.merge(t3,on=['user_id','sku_id'],how='left')
t['f2_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f2_scannums_a_num_user_id_sku_id_sum']
t = t[['user_id','sku_id','f2_average_scan_day_gap']]
sample_user_sku2 = sample_user_sku2.merge(t,on=['user_id','sku_id'],how='left')


###um23
t1 = sample_user_sku2[['user_id','sku_id','f2_attnums_a_num_user_id_sku_id_sum']]
t2 = user_sku2[user_sku2.a_type==2].groupby(['user_id','sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['user_id','sku_id'],how='left')
t3 = user_sku2[user_sku2.a_type==2].groupby(['user_id','sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','a_date':'a_date_min'})
t = t.merge(t3,on=['user_id','sku_id'],how='left')
t['f2_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f2_attnums_a_num_user_id_sku_id_sum']
t = t[['user_id','sku_id','f2_average_att_day_gap']]
sample_user_sku2 = sample_user_sku2.merge(t,on=['user_id','sku_id'],how='left')


###um24
t1 = sample_user_sku2[['user_id','sku_id','f2_commentnums_comment_create_tm_user_id_sku_id_nunique']]
t2 = user_sku2.groupby(['user_id','sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['user_id','sku_id'],how='left')
t3 = user_sku2.groupby(['user_id','sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['user_id','sku_id'],how='left')
t['f2_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f2_commentnums_comment_create_tm_user_id_sku_id_nunique']
t = t[['user_id','sku_id','f2_average_comment_day_gap']]
sample_user_sku2 = sample_user_sku2.merge(t,on=['user_id','sku_id'],how='left')


###u25-u32还没写，正在考虑
user_sku2_a_o_date = user_sku2[((pd.to_datetime(user_sku2.a_date)<=pd.to_datetime(user_sku2.o_date.fillna('2018-01-01'))))]
user_sku2_a_o_date = user_sku2_a_o_date.sort_values(by=['sku_id','a_date','o_date']).drop_duplicates(['sku_id','a_date'],keep="first")
user_sku2_a_o_date = user_sku2_a_o_date[['sku_id','a_date','o_date','cate']]
user_sku2_a_o_date['o_a_gap']=(pd.to_datetime(user_sku2_a_o_date.o_date)-pd.to_datetime(user_sku2_a_o_date.a_date)).dt.days 
FLAG='user_sku2_a_o_'
sample_user_sku2 = feat_count(sample_user_sku2,user_sku2_a_o_date[user_sku2_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG+'order_')
sample_user_sku2 = feat_mean(sample_user_sku2,user_sku2_a_o_date[user_sku2_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG)

###um33
t1 = user_sku2[['user_id','sku_id','o_sku_num','price']] 
t1['user_sku_total_price'] = t1['o_sku_num'] * t1['price']
sample_user_sku2 = feat_sum(sample_user_sku2,t1,['user_id','sku_id'],'user_sku_total_price','f2_user_total_consumption_')

###um34-um36
sample_user_sku2['f2_total_consumption_in_7days_'] = sample_user_sku2['f2_user_total_consumption_user_sku_total_price_user_id_sku_id_sum']/12
sample_user_sku2['f2_total_consumption_in_15days_'] = sample_user_sku2['f2_user_total_consumption_user_sku_total_price_user_id_sku_id_sum']/6
sample_user_sku2['f2_total_consumption_in_31days_'] = sample_user_sku2['f2_user_total_consumption_user_sku_total_price_user_id_sku_id_sum']/3


###um37-um40
sample_user_sku2 = feat_min(sample_user_sku2,user_sku2,['user_id','sku_id'],'o_day','f2_buytime_early_')
sample_user_sku2 = feat_min(sample_user_sku2,user_sku2[user_sku2.a_type==1],['user_id','sku_id'],'a_day','f2_scantime_early_')
sample_user_sku2 = feat_min(sample_user_sku2,user_sku2[user_sku2.a_type==2],['user_id','sku_id'],'a_day','f2_atttime_early_')
sample_user_sku2 = feat_min(sample_user_sku2,user_sku2,['user_id','sku_id'],'comment_day','f2_commenttime_early_')

###um41-um44
sample_user_sku2 = feat_max(sample_user_sku2,user_sku2,['user_id','sku_id'],'o_day','f2_buytime_late_')
sample_user_sku2 = feat_max(sample_user_sku2,user_sku2[user_sku2.a_type==1],['user_id','sku_id'],'a_day','f2_scantime_late_')
sample_user_sku2 = feat_max(sample_user_sku2,user_sku2[user_sku2.a_type==2],['user_id','sku_id'],'a_day','f2_atttime_late_')
sample_user_sku2 = feat_max(sample_user_sku2,user_sku2,['user_id','sku_id'],'comment_day','f2_commenttime_late_')


sample_user_sku2 = sample_user_sku2.drop_duplicates()
gc.collect()

In [ ]:
###用户-商品组合特征
#####dataset1_user_sku
#####
###for dataset1
user_sku1 = feature1[(feature1.cate==30)|(feature1.cate==101)]
t = user_sku1[['user_id','sku_id']]
t.drop_duplicates(inplace=True)
###um1
sample_user_sku1 = feat_nunique(t,user_sku1,['user_id','sku_id'],'o_id','f1_buytimes_')
###um2-um4
sample_user_sku1['f1_buytimes_in_7days_'] = sample_user_sku1['f1_buytimes_o_id_user_id_sku_id_nunique']/12
sample_user_sku1['f1_buytimes_in_15days_'] = sample_user_sku1['f1_buytimes_o_id_user_id_sku_id_nunique']/6
sample_user_sku1['f1_buytimes_in_31days_'] = sample_user_sku1['f1_buytimes_o_id_user_id_sku_id_nunique']/3

###um5
sample_user_sku1 = feat_sum(sample_user_sku1,user_sku1,['user_id','sku_id'],'o_sku_num','f1_buynums_')
###um6-um8
sample_user_sku1['f1_buynums_in_7days_'] = sample_user_sku1['f1_buynums_o_sku_num_user_id_sku_id_sum']/12
sample_user_sku1['f1_buynums_in_15days_'] = sample_user_sku1['f1_buynums_o_sku_num_user_id_sku_id_sum']/6
sample_user_sku1['f1_buynums_in_31days_'] = sample_user_sku1['f1_buynums_o_sku_num_user_id_sku_id_sum']/3

###um9
sample_user_sku1 = feat_sum(sample_user_sku1,user_sku1[user_sku1.a_type==1],['user_id','sku_id'],'a_num','f1_scannums_')
###um10-um12
sample_user_sku1['f1_scannums_in_7days_'] = sample_user_sku1['f1_scannums_a_num_user_id_sku_id_sum']/12
sample_user_sku1['f1_scannums_in_15days_'] = sample_user_sku1['f1_scannums_a_num_user_id_sku_id_sum']/6
sample_user_sku1['f1_scannums_in_31days_'] = sample_user_sku1['f1_scannums_a_num_user_id_sku_id_sum']/3

###um13
sample_user_sku1 = feat_sum(sample_user_sku1,user_sku1[user_sku1.a_type==2],['user_id','sku_id'],'a_num','f1_attnums_')
###um14-um16
sample_user_sku1['f1_attnums_in_7days_'] = sample_user_sku1['f1_attnums_a_num_user_id_sku_id_sum']/12
sample_user_sku1['f1_attnums_in_15days_'] = sample_user_sku1['f1_attnums_a_num_user_id_sku_id_sum']/6
sample_user_sku1['f1_attnums_in_31days_'] = sample_user_sku1['f1_attnums_a_num_user_id_sku_id_sum']/3

###um17
sample_user_sku1 = feat_nunique(sample_user_sku1,user_sku1,['user_id','sku_id'],'comment_create_tm','f1_commentnums_')
###um18-20
sample_user_sku1['f1_commentnums_in_7days_'] = sample_user_sku1['f1_commentnums_comment_create_tm_user_id_sku_id_nunique']/12
sample_user_sku1['f1_commentnums_in_15days_'] = sample_user_sku1['f1_commentnums_comment_create_tm_user_id_sku_id_nunique']/6
sample_user_sku1['f1_commentnums_in_31days_'] = sample_user_sku1['f1_commentnums_comment_create_tm_user_id_sku_id_nunique']/3

###um21
t1 = sample_user_sku1[['user_id','f1_buytimes_o_id_user_id_sku_id_nunique','sku_id']]
t2 = user_sku1.groupby(['user_id','sku_id'])['o_date'].max().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','o_date':'o_date_max'})
t = t2.merge(t1,on=['user_id','sku_id'],how='left')
t3 = user_sku1.groupby(['user_id','sku_id'])['o_date'].min().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','o_date':'o_date_min'})
t = t.merge(t3,on=['user_id','sku_id'],how='left')
t['f1_average_buy_day_gap'] = (t['o_date_max'] - t['o_date_min']).dt.days/t['f1_buytimes_o_id_user_id_sku_id_nunique']
t = t[['user_id','sku_id','f1_average_buy_day_gap']]
sample_user_sku1 = sample_user_sku1.merge(t,on=['user_id','sku_id'],how='left')

###um22
t1 = sample_user_sku1[['user_id','f1_scannums_a_num_user_id_sku_id_sum','sku_id']]
t2 = user_sku1[user_sku1.a_type==1].groupby(['user_id','sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['user_id','sku_id'],how='left')
t3 = user_sku1[user_sku1.a_type==1].groupby(['user_id','sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','a_date':'a_date_min'})
t = t.merge(t3,on=['user_id','sku_id'],how='left')
t['f1_average_scan_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f1_scannums_a_num_user_id_sku_id_sum']
t = t[['user_id','sku_id','f1_average_scan_day_gap']]
sample_user_sku1 = sample_user_sku1.merge(t,on=['user_id','sku_id'],how='left')


###um23
t1 = sample_user_sku1[['user_id','sku_id','f1_attnums_a_num_user_id_sku_id_sum']]
t2 = user_sku1[user_sku1.a_type==2].groupby(['user_id','sku_id'])['a_date'].max().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','a_date':'a_date_max'})
t = t2.merge(t1,on=['user_id','sku_id'],how='left')
t3 = user_sku1[user_sku1.a_type==2].groupby(['user_id','sku_id'])['a_date'].min().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','a_date':'a_date_min'})
t = t.merge(t3,on=['user_id','sku_id'],how='left')
t['f1_average_att_day_gap'] = (t['a_date_max'] - t['a_date_min']).dt.days/t['f1_attnums_a_num_user_id_sku_id_sum']
t = t[['user_id','sku_id','f1_average_att_day_gap']]
sample_user_sku1 = sample_user_sku1.merge(t,on=['user_id','sku_id'],how='left')


###um24
t1 = sample_user_sku1[['user_id','sku_id','f1_commentnums_comment_create_tm_user_id_sku_id_nunique']]
t2 = user_sku1.groupby(['user_id','sku_id'])['comment_create_tm'].max().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','comment_create_tm':'comment_date_max'})
t = t2.merge(t1,on=['user_id','sku_id'],how='left')
t3 = user_sku1.groupby(['user_id','sku_id'])['comment_create_tm'].min().reset_index().rename(columns={'user_id':'user_id','sku_id':'sku_id','comment_create_tm':'comment_date_min'})
t = t.merge(t3,on=['user_id','sku_id'],how='left')
t['f1_average_comment_day_gap'] = (t['comment_date_max'] - t['comment_date_min']).dt.days/t['f1_commentnums_comment_create_tm_user_id_sku_id_nunique']
t = t[['user_id','sku_id','f1_average_comment_day_gap']]
sample_user_sku1 = sample_user_sku1.merge(t,on=['user_id','sku_id'],how='left')


###um25-um32还没写，正在考虑
user_sku1_a_o_date = user_sku1[((pd.to_datetime(user_sku1.a_date)<=pd.to_datetime(user_sku1.o_date.fillna('2018-01-01'))))]
user_sku1_a_o_date = user_sku1_a_o_date.sort_values(by=['sku_id','a_date','o_date']).drop_duplicates(['sku_id','a_date'],keep="first")
user_sku1_a_o_date = user_sku1_a_o_date[['sku_id','a_date','o_date','cate']]
user_sku1_a_o_date['o_a_gap']=(pd.to_datetime(user_sku1_a_o_date.o_date)-pd.to_datetime(user_sku1_a_o_date.a_date)).dt.days 
FLAG='user_sku1_a_o_'
sample_user_sku1 = feat_count(sample_user_sku1,user_sku1_a_o_date[user_sku1_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG+'order_')
sample_user_sku1 = feat_mean(sample_user_sku1,user_sku1_a_o_date[user_sku1_a_o_date.o_a_gap>=-1],['sku_id'],'o_a_gap',FLAG)

###um33
t1 = user_sku1[['user_id','sku_id','o_sku_num','price']] 
t1['user_sku_total_price'] = t1['o_sku_num'] * t1['price']
sample_user_sku1 = feat_sum(sample_user_sku1,t1,['user_id','sku_id'],'user_sku_total_price','f1_user_total_consumption_')

###um34-um36
sample_user_sku1['f1_total_consumption_in_7days_'] = sample_user_sku1['f1_user_total_consumption_user_sku_total_price_user_id_sku_id_sum']/12
sample_user_sku1['f1_total_consumption_in_15days_'] = sample_user_sku1['f1_user_total_consumption_user_sku_total_price_user_id_sku_id_sum']/6
sample_user_sku1['f1_total_consumption_in_31days_'] = sample_user_sku1['f1_user_total_consumption_user_sku_total_price_user_id_sku_id_sum']/3


###um37-um40
sample_user_sku1 = feat_min(sample_user_sku1,user_sku1,['user_id','sku_id'],'o_day','f1_buytime_early_')
sample_user_sku1 = feat_min(sample_user_sku1,user_sku1[user_sku1.a_type==1],['user_id','sku_id'],'a_day','f1_scantime_early_')
sample_user_sku1 = feat_min(sample_user_sku1,user_sku1[user_sku1.a_type==2],['user_id','sku_id'],'a_day','f1_atttime_early_')
sample_user_sku1 = feat_min(sample_user_sku1,user_sku1,['user_id','sku_id'],'comment_day','f1_commenttime_early_')

###um41-um44
sample_user_sku1 = feat_max(sample_user_sku1,user_sku1,['user_id','sku_id'],'o_day','f1_buytime_late_')
sample_user_sku1 = feat_max(sample_user_sku1,user_sku1[user_sku1.a_type==1],['user_id','sku_id'],'a_day','f1_scantime_late_')
sample_user_sku1 = feat_max(sample_user_sku1,user_sku1[user_sku1.a_type==2],['user_id','sku_id'],'a_day','f1_atttime_late_')
sample_user_sku1 = feat_max(sample_user_sku1,user_sku1,['user_id','sku_id'],'comment_day','f1_commenttime_late_')


sample_user_sku1 = sample_user_sku1.drop_duplicates()
gc.collect()

In [ ]:
####最后把三个day的特征加上，之前加会有memoryerror
####最后把三个day的特征加上，之前加会有memoryerror
####最后把三个day的特征加上，之前加会有memoryerror

sample3 = sample_user_sku3.merge(sample_sku3,on=['sku_id'],how='left')
sample3 = sample3.merge(sample_user3,on=['user_id'],how='left')
test_cv = sample3[['user_id']]

sample2 = sample_user_sku2.merge(sample_sku2,on=['sku_id'],how='left')
sample2 = sample2.merge(sample_user2,on=['user_id'],how='left')

sample1 = sample_user_sku1.merge(sample_sku1,on=['sku_id'],how='left')
sample1 = sample1.merge(sample_user1,on=['user_id'],how='left')

print(sample3.shape,sample2.shape,sample1.shape)

In [17]:
gc.collect()
train_dataset2 = sample2.merge(dataset2,on=['user_id','sku_id'],how='left')
train_dataset1 = sample1.merge(dataset1,on=['user_id','sku_id'],how='left')
train_dataset2 = train_dataset2.drop(['user_id','sku_id'],axis=1)
train_dataset1 = train_dataset1.drop(['user_id','sku_id'],axis=1)
print(train_dataset2.shape,train_dataset1.shape)
gc.collect()

(1389415, 117) (1347333, 117)


14

In [18]:
train_dataset2 = train_dataset2.fillna(0)
train_dataset1 = train_dataset1.fillna(0)

In [19]:
feature_combine = sample1.drop(['user_id','sku_id'],axis=1)
feature_list = feature_combine.columns.tolist()
# train_feature_label, val_feature_label
# test_feature
print (len(feature_list))

115


In [23]:
sample3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1282992 entries, 0 to 1282991
Columns: 117 entries, user_id to comment_day
dtypes: float64(96), int64(21)
memory usage: 1.1 GB


In [24]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

model = SBBTree(params=params, stacking_num=5, bagging_num=3,  bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)

# train 下个月购买次数预测 回归模型

train_data = train_dataset2.drop(['label_1','label_2'],axis=1)
train_label = train_dataset2['label_1']

X = train_data.values 
y = train_label.values

X_pred = sample3.drop(['user_id','sku_id'],axis=1).values

model.fit(X,y)
test_cv['probility'] = model.predict(X_pred)

MemoryError: 

In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
model = SBBTree(params=params, stacking_num=5, bagging_num=3,  bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)

# train 当月首次购买时间预测 回归模型
train_data = train_dataset2.drop(['label_1','label_2'],axis=1)
train_label = train_dataset2['label_2']

X = train_data.values 
y = train_label.values

X_pred = sample3.drop(['user_id','sku_id'],axis=1).values

model.fit(X,y)
test_cv['buy_gap'] = model.predict(X_pred)